# Статанализ масок облаков и снега

В документе исследуется вопрос о том, как построить маски облачности и снега, пользуясь статистическими методами. Здесь были использованы выборки с двух сцен Landsat-8 и размеченные данные по трем классам: облако, снег, остальное. Документ состоит из нескольких связанных разделов.

 * В разделе [Предобработка](#Preprocessing) дается ссылка на исходные данные и производится очистка данных от явно ошибочных значений.
 * В разделе [Первый взгляд на данные](#Exploration) производится разведочный анализ данных. Раздел состоит из следующих частей:
   - [Отражающая способность различных объектов](#Reflectance): здесь рассматривается вопрос о том, какими спектральными характеристиками обладает каждый из интересующих классов.
   - [Зависимость от сцены](#Reflectance-Scene): здесь исследуюется вопрос о том, в каких каналах вариативность данных в зависимости от сцены высока, а в каких нет.
   - [Вывод](#Exploration-Resume): сводка основных полученных результатов.
 * Раздел [Простая модель](#SimpleModel) посвящен разработке статистических моделей для классификации. Здесь содержатся следующие подразделы:
   - [Выбор объясняющих переменных](#CovariatesSelection): в этом подразделе производится визуальная оценка тех каналов, которые потенциально будут наиболее полезными при построении модели.
   - [Фиктивные переменные](#DummyVars): строятся фиктивные переменные, которые будут использоваться при построении моделей.
   - [Построение моделей](#Modelling): в разделе создаются различного рода статмодели и исследуется их качество. Это наиболее обширный подраздел во всем документе и он делится на несколько секций. В первых секциях исследуются модели на базе логистической регрессии. Затем рассматриваются аддитивные модели.
   Все секции устроены одинаково, но логика и ход создания модели подробно описаны только для первой из них (облачность). Вторая секция (снег) в основном следует описанной логике и подробности в ней опускаются.
     + [ЛР модель для облаков](#Clouds)
       * [Построение собственно модели](#CloudsModel)
       * [Валидация модели](#CloudsValidation)
       * [Вывод по модели](#CloudsResume)
     + [ЛР модель для снега](#Snow)
       * [Модель-1](#SnowModel1)
       * [Валидация модели-1](#SnowValidation1)
       * [Модель-2](#SnowModel2)
       * [Валидация модели-2](#SnowValidation2)
       * [Модель-3](#SnowModel3)
       * [Валидация модели-3](#SnowValidation3)
       * [Вывод по модели](#SnowResume)


<a id='Preprocessing'></a>
## Предобработка 

В файл были экспортированы обучающие данные по облачности и снегу (см. https://176.9.38.120/cruncher/notebooks/deforestation/CMask_trainings.ipynb).

In [1]:
points = read.table('data/CMask_trainings/random_points.csv', header=TRUE, sep=';')

# summary(points)

Содержимое колонок:

  * cat -- идентификатор точки, в которой производились измерения;
  * l1_b1 -- l1_b11 -- данные LANDSAT8 (после атмосферной коррекции) по сцене LC81120282015365LGN00;
  * l2_b1 -- l2_b11 -- данные LANDSAT8 (после атмосферной коррекции) по сцене LC81130272015356LGN00;
  * t1, t2 -- данные по наземным объектам сцен LC81120282015365LGN00 и LC81130272015356LGN00 соответственно, значения: 1-облака;2-снег;3-остальное.
  * x, y -- координаты (UTM);

Соберем данные в dataframe для удобства анализа так, чтобы в нем одна строка соответствовала одному измерению, а не двум, как сейчас:

In [2]:
tmp1 = points[c('cat', 'x', 'y', 't1',
                'l1_b1', 'l1_b2', 'l1_b3', 'l1_b4', 'l1_b5', 'l1_b6', 
                'l1_b7', 'l1_b8', 'l1_b9', 'l1_b10', 'l1_b11'
               )]
names(tmp1) = c('cat', 'x', 'y', 't',
                'b1', 'b2', 'b3', 'b4', 'b5', 'b6', 'b7', 'b8', 'b9', 'b10', 'b11'
               )

tmp1$scene = 'S1' # 'LC81120282015365LGN00'

tmp2 = points[c('cat', 'x', 'y', 't2',
                'l2_b1', 'l2_b2', 'l2_b3', 'l2_b4', 'l2_b5', 'l2_b6', 
                'l2_b7', 'l2_b8', 'l2_b9', 'l2_b10', 'l2_b11'
               )]
names(tmp2) = c('cat', 'x', 'y', 't',
                'b1', 'b2', 'b3', 'b4', 'b5', 'b6', 'b7', 'b8', 'b9', 'b10', 'b11'
               )
tmp2$scene = 'S2' # 'LC81130272015356LGN00'


points = rbind(tmp1, tmp2)

points$t[points$t==0] = NA

points$scene = factor(points$scene)
points$t = factor(points$t)
levels(points$t) <- list(Cloud=1, Snow=2, Other=3)

points = na.omit(points)

summary(points)

      cat              x                y               t        
 Min.   :    1   Min.   :434113   Min.   :4979683   Cloud: 1579  
 1st Qu.:12508   1st Qu.:496221   1st Qu.:5060181   Snow : 3644  
 Median :24998   Median :536605   Median :5117104   Other:45316  
 Mean   :24998   Mean   :541089   Mean   :5112347                
 3rd Qu.:37496   3rd Qu.:586255   3rd Qu.:5165598                
 Max.   :50000   Max.   :664168   Max.   :5215814                
       b1               b2                b3                b4         
 Min.   :0.1124   Min.   :0.08453   Min.   :0.04529   Min.   :0.02953  
 1st Qu.:0.2011   1st Qu.:0.16832   1st Qu.:0.11290   1st Qu.:0.09587  
 Median :0.2291   Median :0.19891   Median :0.14421   Median :0.13350  
 Mean   :0.2529   Mean   :0.22642   Mean   :0.17388   Mean   :0.17051  
 3rd Qu.:0.2732   3rd Qu.:0.24862   3rd Qu.:0.19710   3rd Qu.:0.19912  
 Max.   :0.9407   Max.   :1.07539   Max.   :1.17382   Max.   :1.45950  
       b5               b6        

Видим, что в 9-м канале после атмосферной коррекции получились отрицательные значения. Объясняется это, по всей видимости, тем, что используемые в ходе коррекции константы (gain или bias, см. метаданные) содержали неподходящие для данной сцены значения. Аналогично, почти во всех каналах значения отражающей способности оказались выше 1.

In [3]:
# Число примеров, оставшихся после удаления отсутсвующих данных
dim(points)

[1] 50539    16

In [4]:
points$b8[points$b9 < 0] = 0.0

points$b2[points$b2 > 1] = 1.0
points$b3[points$b3 > 1] = 1.0
points$b4[points$b4 > 1] = 1.0
points$b5[points$b5 > 1] = 1.0
points$b8[points$b8 > 1] = 1.0

<a id='Exploration'></a>
## Первый взгляд на данные

<a id='Reflectance'></a>
### Отражающая способность различных объектов

Построим ящики с усами по каждому каналу и сравним какие значения отражающей способности типичны для каждого типа поверхности.

In [5]:
opar = par(no.readonly = TRUE)      # make a copy of current settings

png("Img/CMASK/type_vs_bands.png", width=728, height=728, units="px")
    par(mfrow = c(3,4))
    boxplot(b1 ~ t, data=points, ylab='b1')
    boxplot(b2 ~ t, data=points, ylab='b2')
    boxplot(b3 ~ t, data=points, ylab='b3')
    boxplot(b4 ~ t, data=points, ylab='b4')

    boxplot(b5 ~ t, data=points, ylab='b5')
    boxplot(b6 ~ t, data=points, ylab='b6')
    boxplot(b7 ~ t, data=points, ylab='b7')
    boxplot(b8 ~ t, data=points, ylab='b8')

    boxplot(b9 ~ t, data=points, ylab='b9')
    boxplot(b10 ~ t, data=points, ylab='b10')
    boxplot(b11 ~ t, data=points, ylab='b11')
    par(opar)
dev.off()

png 
  2

<img src="https://176.9.38.120/cruncher/files/deforestation/Img/CMASK/type_vs_bands.png">

* 1-й--5-й и 8-й каналы ведут себя примерно одинаково: яркости пикслеей, соотвествующие снегу, выше, чем яркости облачных пикселей. А яркость пикселей облаков и снега обычно выше, чем пикселей других объектов. Однако, нельзя провести четкой границы -- области яркостей пикселей различных объектов довольно сильно пересекаются. Видно, что в этих каналах распределение имеет длинный правый хвост: на графиках видны выбросы в верхней части ящиков. Возможно, что стоит преобразовать каналы, чтобы получить более симметричное распределение. Поскольку восьмой канал - панхроматический, то он несет в основном пространственную информацию, а ничего нового с точки зрения спектральной информации не привносит. Поэтому в дальнейшем он буде опущен при анализе.
* 6-й--7-й каналы: высокое отражение в облачных областях, чуть меньшее в заснеженных и еще меньше для остальных объектов. Точно также данные показывают длинный правый хвост в распределении.
* 9-й канал: очень длинный правый хвост в облачных пикселях. Значения облачных пикселей в целом значительно выше, чем у остальных. Поскольку этот канал был разработан для исследования атмосферных эффектов, то, возможно, этот канал будет одним из основных маркеров облако/не облако.
* 10-й и 11-й каналы (температура) cхожи между собой: на них заметно, что температура облаков обычно выше, чем остальных объектов (зима?), хотя у облаков бросается в глаза очень длинный левый хвост распределения.

Преобразуем данные, чтобы получить более равномерные распределения. При этом выкинем из анализа 8-й канал, основную часть каналов прологарифмируем для избавления от длинных хвостов, 9-й канал возведем в степень, а 10-й и 11-й каналы оставим без изменений (длинный хвост только у облаков, а не у всего канала):

In [6]:
png("Img/CMASK/type_vs_logbands.png", width=728, height=728, units="px")
    par(mfrow = c(2, 4))
    boxplot(log(b1) ~ t, data=points, ylab='log(b1')
    boxplot(log(b2) ~ t, data=points, ylab='log(b2')
    boxplot(log(b3) ~ t, data=points, ylab='log(b3')
    boxplot(log(b4) ~ t, data=points, ylab='log(b4')

    boxplot(log(b5) ~ t, data=points, ylab='log(b5)')
    boxplot(log(b6) ~ t, data=points, ylab='log(b6)')
    boxplot(log(b7) ~ t, data=points, ylab='log(b7)')

    boxplot(I(b9^0.5) ~ t, data=points, ylab='sqrt(b9)')
    par(opar)
dev.off()

png 
  2

<img src="https://176.9.38.120/cruncher/files/deforestation/Img/CMASK/type_vs_logbands.png">

Видно, что распределения частично выровнялись, хотя местами по-прежнему остались длинные хвосты.

<a id='Reflectance-Scene'></a>
### Зависимость от сцены

Построим те же самые ящики с усами, но попробуем понять, насколько отличаются данные не только из-за объектов, но и из-за того, что они представлены в разных сценах.

In [7]:
png("Img/CMASK/scene_vs_logbands.png", width=728, height=728, units="px")
    par(mfrow = c(3, 4))
    boxplot(log(b1) ~ scene, data=points, ylab='log(b1)')
    boxplot(log(b2) ~ scene, data=points, ylab='log(b2)')
    boxplot(log(b3) ~ scene, data=points, ylab='log(b3)')
    boxplot(log(b4) ~ scene, data=points, ylab='log(b4)')

    boxplot(log(b5) ~ scene, data=points, ylab='log(b5)')
    boxplot(log(b6) ~ scene, data=points, ylab='log(b6)')
    boxplot(log(b7) ~ scene, data=points, ylab='log(b7)')

    boxplot(I(b9^0.5) ~ scene, data=points, ylab='sqrt(b9)')

    boxplot(b10 ~ scene, data=points, ylab='b10')
    boxplot(b11 ~ scene, data=points, ylab='b11')

    par(opar)
dev.off()

png 
  2

<img src="https://176.9.38.120/cruncher/files/deforestation/Img/CMASK/scene_vs_logbands.png">

Итак, мы видим, что сцена 2 (LC81130272015356LGN00) более яркая, чем сцена 1 (LC81120282015365LGN00) во всех каналах. Вопрос в том, насколько велика эта вариативность данных, которая вносится разными условиями съемки, и не может ли оказаться так, что условия съемки влияют на яркость пикселей гораздо больше, чем тип обеъкта (см., например, различия якркостей в 10-м и 11-м каналах).

Для того, чтобы ответить на этот вопрос построим ящики для отдельных каналов.

#### Канал B1

В первом канале видно, что яркость снега может меняться в достаточно широких пределах в зависимости от сцены, настолько, что она яркости облаков и снежных участков могут пересекаться в большой степени.

In [8]:
png("Img/CMASK/scene_type_vs_b1.png", width=728, height=728, units="px")
    boxplot(log(b1) ~ scene+t, data=points, ylab='log(b1)')
dev.off()

png 
  2

<img src="https://176.9.38.120/cruncher/files/deforestation/Img/CMASK/scene_type_vs_b1.png">

#### Канал B2

Аналогичная картина и во втором канале.

In [9]:
png("Img/CMASK/scene_type_vs_b2.png", width=728, height=728, units="px")
    boxplot(log(b2) ~ scene+t, data=points, ylab='log(b2)')
dev.off()

png 
  2

<img src="https://176.9.38.120/cruncher/files/deforestation/Img/CMASK/scene_type_vs_b2.png">

#### Канал B3

Аналогично и в третьем:

In [10]:
png("Img/CMASK/scene_type_vs_b3.png", width=728, height=728, units="px")
    boxplot(log(b3) ~ scene+t, data=points, ylab='log(b3)')
dev.off()

png 
  2

<img src="https://176.9.38.120/cruncher/files/deforestation/Img/CMASK/scene_type_vs_b3.png">

#### Канал B4

In [11]:
png("Img/CMASK/scene_type_vs_b4.png", width=728, height=728, units="px")
    boxplot(log(b4) ~ scene+t, data=points, ylab='log(b4)')
dev.off()

png 
  2

<img src="https://176.9.38.120/cruncher/files/deforestation/Img/CMASK/scene_type_vs_b4.png">

#### Канал B5

In [12]:
png("Img/CMASK/scene_type_vs_b5.png", width=728, height=728, units="px")
    boxplot(log(b5) ~ scene+t, data=points, ylab='log(b5)')
dev.off()

png 
  2

<img src="https://176.9.38.120/cruncher/files/deforestation/Img/CMASK/scene_type_vs_b5.png">

#### Канал B6

Зато 6-й канал, похоже, меньше зависит от сцены, чем предыдущие:

In [13]:
png("Img/CMASK/scene_type_vs_b6.png", width=728, height=728, units="px")
    boxplot(log(b6) ~ scene+t, data=points, ylab='log(b6)')
dev.off()

png 
  2

<img src="https://176.9.38.120/cruncher/files/deforestation/Img/CMASK/scene_type_vs_b6.png">

#### Канал B7

Очень похож на 6-й канал, но вариативность данных внутри классов в нем чуть меньше.

In [14]:
png("Img/CMASK/scene_type_vs_b7.png", width=728, height=728, units="px")
    boxplot(log(b7) ~ scene+t, data=points, ylab='log(b7)')
dev.off()

png 
  2

<img src="https://176.9.38.120/cruncher/files/deforestation/Img/CMASK/scene_type_vs_b7.png">

#### Канал B9

В 9-м канале вариативность яркостей облачных пикселей между сценами очень велика. Похоже, что от сцены тут зависит больше, чем от типа объекта.

In [15]:
png("Img/CMASK/scene_type_vs_b9.png", width=728, height=728, units="px")
    boxplot(I(b9^0.5) ~ scene+t, data=points, ylab='sqrt(b9)')
dev.off()

png 
  2

<img src="https://176.9.38.120/cruncher/files/deforestation/Img/CMASK/scene_type_vs_b9.png">

#### Канал B10

Аналогичная проблема облачных пикселей и в 10-м канале: от сцены зависит больше, чем от типа объекта, особенно ярко это проявляется для облаков.

In [16]:
png("Img/CMASK/scene_type_vs_b10.png", width=728, height=728, units="px")
    boxplot(b10 ~ scene+t, data=points, ylab='b10')
dev.off()

png 
  2

<img src="https://176.9.38.120/cruncher/files/deforestation/Img/CMASK/scene_type_vs_b10.png">

#### Канал B11

Та же самая проблема: от сцены зависит больше, чем от типа объекта.

In [17]:
png("Img/CMASK/scene_type_vs_b11.png", width=728, height=728, units="px")
    boxplot(b11 ~ scene+t, data=points, ylab='b11')
dev.off()

png 
  2

<img src="https://176.9.38.120/cruncher/files/deforestation/Img/CMASK/scene_type_vs_b11.png">

<a id='Exploration-Resume'></a>
### Вывод

 * Для различения облаков и снега, видимо, необходимо использовать каналы 1--7. В каналах 9--11 также содержится полезная информация, но воспользоваться ей будет сложнее из-за большей вариативности из-за условий съемки.
 * Каналы 1--5 и каналы 6--7 образуют две группы переменных. Переменные в каждой группе ведут себя схожим образом (поведение каналов 1--5 очень похоже между собой для выбранных классов; аналогично с каналами 6 и 7). Поскольку многие методы очень чувствительны к корреляции объясняющих переменных, то есть смысл подумать о том, чтобы выбрать по одному каналу из каждой группы. Возможно такое уменьшение числа переменных (но и корреляции между ними) даст положительный эффект на точность распознавания.

<a id='SimpleModel'></a>
## Простая модель

В этом разделе построим простейшую модель на основе логистической регрессии и посмотрим, насколько она решает задачу разделения интересующих нас объектов. Пойдем по следующей схеме работы:

1. Предварительный выбор объясняющих переменных. Объясняющих переменных (каналов, в нашем случае) много, но они сильно коррелируют между собой. Для большей устойчивости решения выберем несколько переменных, остальные пока проигнорируем (на следующем этапе оценим правильность выбора переменных).
2. Создание фиктивных переменных. Поскольку у нас выделено три объекта (класс "все остальное" назовем условно одним объектом), и они не упорядочены, то нам понадобится создать фиктивные переменные, которые мы будем использовать.
3. Построение моделей, отражающих связь между объясняющими переменными и зависимыми.
4. Исследуем модели, проверив, выполняются ли условия, на которые опирается процедура построения логистической регрессии.
5. Посмотрим, насколько изменится качество модели, если выбрать другой набор объясняющих переменных.

<a id='CovariatesSelection'></a>
### Предварительный выбор объясняющих переменных

Посмотрим еще раз на взаимную корреляциию объясняющих переменных (для ускорения отрисовки сделаем небольшую подвыборку данных). 

In [18]:
psample_ind = sample(1:length(points[, 1]), size=500)
psample = points[psample_ind, ]

png("Img/CMASK/all_bands.png", width=728, height=728, units="px")
  pairs(~log(b1)+log(b2)+log(b3)+log(b4)+log(b5)+log(b6)+log(b7)+sqrt(b9)+b10+b11, data=psample, pch=19, cex=0.1) 
dev.off()

png 
  2

<img src="https://176.9.38.120/cruncher/files/deforestation/Img/CMASK/all_bands.png">

Итак, мы видим, что очень сильна корреляция между 6-м и 7-м каналами, затем между 10-м и 11-м каналами, а также между 1-м, 2-м, 3-м и 4-м каналами, на эти 4 канала довольно похож 5-й, но все-таки, видимо, его следует выделить в отдельную группу. Ту же самую картину мы увидим, если расчитаем коэффициент корреляции между этими переменными:

In [19]:
cor(data.frame(log(points$b1), log(points$b2), log(points$b3), log(points$b4), log(points$b5),
    log(points$b6), log(points$b7), sqrt(points$b9+0.001), points$b10, points$b11))

,log.points.b1.,log.points.b2.,log.points.b3.,log.points.b4.,log.points.b5.,log.points.b6.,log.points.b7.,sqrt.points.b9...0.001.,points.b10,points.b11
log.points.b1.,1.00000000,0.99643580,0.97639013,0.95335102,0.77517809,0.56200970,0.62302423,0.29965771,0.01198254,0.02497259
log.points.b2.,0.99643580,1.00000000,0.99003616,0.97372490,0.81425120,0.60139706,0.66104285,0.30740816,0.01601809,0.02772027
log.points.b3.,0.97639013,0.99003616,1.00000000,0.99513008,0.87490047,0.66889060,0.72503729,0.32957731,0.03250238,0.03846062
log.points.b4.,0.95335102,0.97372490,0.99513008,1.00000000,0.90503815,0.71378830,0.76718334,0.32889596,0.04236531,0.04731348
log.points.b5.,0.7751781,0.8142512,0.8749005,0.9050381,1.0000000,0.8445939,0.8719638,0.3426454,0.1084817,0.1027279
log.points.b6.,0.56200970,0.60139706,0.66889060,0.71378830,0.84459391,1.00000000,0.99188780,0.44287939,0.08266380,0.06304586
log.points.b7.,0.62302423,0.66104285,0.72503729,0.76718334,0.87196375,0.99188780,1.00000000,0.44975283,0.05640044,0.03886842
sqrt.points.b9...0.001.,0.2996577,0.3074082,0.3295773,0.3288960,0.3426454,0.4428794,0.4497528,1.0000000,-0.3242166,-0.4123782
points.b10,0.01198254,0.01601809,0.03250238,0.04236531,0.10848175,0.08266380,0.05640044,-0.32421662,1.00000000,0.98045369
points.b11,0.02497259,0.02772027,0.03846062,0.04731348,0.10272787,0.06304586,0.03886842,-0.41237817,0.98045369,1.00000000


<a id='DummyVars'></a>
### Фиктивные переменные

Возможно, есть хороший способ создания фиктивных переменных вместо одной зависимой переменной, но я всю процедуру сделаю руками: введу три новых переменных answ1, answ2, answ3 которые будут кодировать тот факт, что пиксель принадлежит облачному, снежному или иному участку снимка соответственно. Для этого создам новый набор данных, в котором будут содержаться только интересующие нас переменные.

In [20]:
pdata = data.frame(answ1=(points$t=='Cloud'), answ2=(points$t=='Snow'), answ3=(points$t=='Other'), 
                   b1=points$b1, b2=points$b2, b3=points$b3, b4=points$b4, b5=points$b5, b6=points$b6, 
                   b7=points$b7, b9=points$b9, b10=points$b10, b11=points$b11, scene=points$scene
                  )

<a id='Modelling'></a>
### Построение моделей

Построим три модели (по числу фиктивных переменных) на базе логистической регрессии. Также не будем использовать преобразование переменных на этом этапе построения моделей. Начанем с определения того, что пиксель является облаком.

<a id='Clouds'></a>
#### Облако-не-облако

<a id='CloudsModel'></a>
##### Модель

Построим модель определяющую, явлеяется ли данный пиксель облаком и найдем те каналы, которые дают наибольшее качество модели (как отмечалось выше, среди имеющихся объясняющих переменных очень высокая корреляция). Для начала построим модель со всеми имеющимися переменными, включая название сцены.

In [21]:
simple.glm1 = glm(answ1~b1 + b2 + b3 + b4 + b5 + b6 +b7 + b9 + b10 + b11 + scene, data=pdata, family=binomial)
summary(simple.glm1)

Warning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurred


Call:
glm(formula = answ1 ~ b1 + b2 + b3 + b4 + b5 + b6 + b7 + b9 + 
    b10 + b11 + scene, family = binomial, data = pdata)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-4.3471  -0.0178  -0.0075  -0.0043   3.7550  

Coefficients:
            Estimate Std. Error z value Pr(>|z|)    
(Intercept)  -9.2134    12.6052  -0.731  0.46483    
b1            3.8466    36.9262   0.104  0.91703    
b2           14.6088    37.8190   0.386  0.69929    
b3          -99.3805    39.7492  -2.500  0.01241 *  
b4           85.4198    27.3904   3.119  0.00182 ** 
b5          -37.1751     3.9036  -9.523  < 2e-16 ***
b6          -10.9699    12.0687  -0.909  0.36337    
b7          112.6315    14.7005   7.662 1.83e-14 ***
b9           52.7175     8.2705   6.374 1.84e-10 ***
b10          -0.1721     0.1278  -1.347  0.17794    
b11           0.1736     0.1287   1.349  0.17749    
sceneS2       4.5265     0.5016   9.023  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’

Оценкой качества модели может служить несколько параметров, например отклонения (остатки, Deviance) прогнозируемых величин от тех, что зафиксированы на самом деле, другая оценка качества - Akaike Information Criterion (AIC), в котором учитывается как качество подгонки модели, так и ее сложность.  Если оринтироваться на Akaike Information Criterion, то AIC коэффициент у simple.glm2 равен 1313.2 (чем меньше, тем лучше). Но мы видим, что часть коэффициентов регрессии статистически не значимы.

Последовательно будем избавляться от "лишних" коэффициентов регрессии. Для этого в цикле будем удалять по одному коэффициенту и оценивать, как изменился результат. Качество будем оценивать по отклонениям прогнозных значений от реальных. Однако, поскольку у полной модели больше коэффициентов в уравнении регрессии, то и отклонения полной модели будут меньше. Поэтому значимость определенного коэффициента, будем определять по следующей схеме: 
 * выкинем одну переменную, например, b1;
 * очевидно, что в модели без этой переменной (т.е. с коэффициентом b1 == 0) отклонения должны быть больше, чем в модели с этим коэффициентом, поскольку в полной модели $p_2 = 11$ параметров, а в "урезанной" модели $p_1=10$ параметров;
 * соответственно мы можем выдвинуть гипотезу, что коэффициент перед b1 равен нулю и проверить ее, сравнив величины отклонений этих двух моделей: чем больше разность отклонений моделей, тем больше вероятность того, что коэффициент перед b1 ненулевой;
 * разность отклонений двух моделей ассимтотически распределена как Хи-квадрат с $p_2 - p_1 = 1$ степенями свободы: $D_1 - D_1 \sim X^2_{p_2 - p_1}$
 * пробежавшись по всем переменным мы можем определить значимость каждой из них.
 
Эту процедуру автоматизирует функция drop1. Воспользуемся ей:

In [22]:
drop1(simple.glm1, test="Chi")

Warning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurred

,Df,Deviance,AIC,LRT,Pr(>Chi)
<none>,NA,1289.236,1313.236,NA,NA
b1,1.000000e+00,1.289247e+03,1.311247e+03,1.085342e-02,9.170267e-01
b2,1.0000000,1289.3857572,1311.3857572,0.1494317,0.6990790
b3,1.000000e+00,1.295625e+03,1.317625e+03,6.388735e+00,1.148469e-02
b4,1.000000e+00,1.299156e+03,1.321156e+03,9.919714e+00,1.635178e-03
b5,1.000000e+00,1.394102e+03,1.416102e+03,1.048654e+02,1.307192e-24
b6,1.0000000,1290.0648232,1312.0648232,0.8284977,0.3627073
b7,1.000000e+00,1.353729e+03,1.375729e+03,6.449295e+01,9.687858e-16
b9,1.000000e+00,1.336635e+03,1.358635e+03,4.739900e+01,5.791226e-12
b10,1.0000000,1291.0570998,1313.0570998,1.8207743,0.1772211


Итого, у модели, содержащей все переменные, остатки (Deviance) равняются 1289.236. Если мы викинем b1, то остатки будут равны 1.289247e+03(=1289.247), разность остатков составляет 1.085342e-02=0.01085, соотвественно p-значение распределения Хи-квадрат с одной степенью свободы равно 9.170267e-01, что явно очень много и мы можем выкинуть эту переменную безбоязненно.

Повторяем данную процедуру для нового набора переменных:

In [23]:
simple.glm2 = glm(answ1~ b2 + b3 + b4 + b5 + b6 +b7 + b9 + b10 + b11 + scene, data=pdata, family=binomial)
summary(simple.glm2)
drop1(simple.glm2, test="Chi")

Warning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurred


Call:
glm(formula = answ1 ~ b2 + b3 + b4 + b5 + b6 + b7 + b9 + b10 + 
    b11 + scene, family = binomial, data = pdata)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-4.3477  -0.0178  -0.0075  -0.0043   3.7519  

Coefficients:
            Estimate Std. Error z value Pr(>|z|)    
(Intercept)  -9.1119    12.5661  -0.725 0.468381    
b2           18.2279    14.9870   1.216 0.223889    
b3          -97.4374    35.1160  -2.775 0.005525 ** 
b4           83.7884    22.4958   3.725 0.000196 ***
b5          -37.2842     3.7608  -9.914  < 2e-16 ***
b6          -10.7796    11.9278  -0.904 0.366136    
b7          112.3931    14.5178   7.742 9.81e-15 ***
b9           52.7780     8.2485   6.399 1.57e-10 ***
b10          -0.1730     0.1275  -1.357 0.174754    
b11           0.1745     0.1284   1.360 0.173874    
sceneS2       4.5337     0.4969   9.124  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family take

Warning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurred

,Df,Deviance,AIC,LRT,Pr(>Chi)
<none>,NA,1289.247,1311.247,NA,NA
b2,1.000000,1290.736493,1310.736493,1.489314,0.222323
b3,1.000000e+00,1.296720e+03,1.316720e+03,7.472685e+00,6.264206e-03
b4,1.000000e+00,1.302318e+03,1.322318e+03,1.307111e+01,2.998858e-04
b5,1.000000e+00,1.403461e+03,1.423461e+03,1.142136e+02,1.170027e-26
b6,1.0000000,1290.0659227,1310.0659227,0.8187438,0.3655477
b7,1.000000e+00,1.354922e+03,1.374922e+03,6.567495e+01,5.317747e-16
b9,1.000000e+00,1.337127e+03,1.357127e+03,4.787991e+01,4.531409e-12
b10,1.0000000,1291.0949258,1311.0949258,1.8477469,0.1740457
b11,1.0000000,1291.1067768,1311.1067768,1.8595978,0.1726712


AIC улучшилось, но остаются еще незначимые коэффициенты. Выкидываем b6 и продолжаем:

In [24]:
simple.glm3 = glm(answ1~ b2 + b3 + b4 + b5 +b7 + b9 + b10 + b11 + scene, data=pdata, family=binomial)
summary(simple.glm3)
drop1(simple.glm3, test="Chi")

Warning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurred


Call:
glm(formula = answ1 ~ b2 + b3 + b4 + b5 + b7 + b9 + b10 + b11 + 
    scene, family = binomial, data = pdata)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-4.3163  -0.0181  -0.0077  -0.0044   3.7809  

Coefficients:
            Estimate Std. Error z value Pr(>|z|)    
(Intercept)  -7.0150    12.3120  -0.570 0.568830    
b2           19.0142    14.9371   1.273 0.203035    
b3          -96.5989    35.0669  -2.755 0.005875 ** 
b4           83.9253    22.4924   3.731 0.000191 ***
b5          -38.2918     3.5976 -10.644  < 2e-16 ***
b7           99.8653     4.0618  24.586  < 2e-16 ***
b9           51.7107     8.1173   6.370 1.88e-10 ***
b10          -0.1684     0.1272  -1.324 0.185347    
b11           0.1612     0.1273   1.266 0.205495    
sceneS2       4.4812     0.4922   9.104  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 14053.6  on 50538  degrees 

Warning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurred

,Df,Deviance,AIC,LRT,Pr(>Chi)
<none>,NA,1290.066,1310.066,NA,NA
b2,1.0000000,1291.6942095,1309.6942095,1.6282868,0.2019402
b3,1.000000e+00,1.297433e+03,1.315433e+03,7.366960e+00,6.643313e-03
b4,1.000000e+00,1.303180e+03,1.321180e+03,1.311393e+01,2.931089e-04
b5,1.000000e+00,1.424451e+03,1.442451e+03,1.343853e+02,4.499798e-31
b7,1.000,4899.839,4917.839,3609.773,0.000
b9,1.000000e+00,1.337128e+03,1.355128e+03,4.706173e+01,6.878583e-12
b10,1.000000,1291.825125,1309.825125,1.759202,0.184724
b11,1.000000,1291.676106,1309.676106,1.610184,0.204466
scene,1.000000e+00,1.396101e+03,1.414101e+03,1.060354e+02,7.243042e-25


Выкидываем b11:

In [25]:
simple.glm4 = glm(answ1~ b2 + b3 + b4 + b5 +b7 + b9 + b10 + scene, data=pdata, family=binomial)
summary(simple.glm4)
drop1(simple.glm4, test="Chi")

Warning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurred


Call:
glm(formula = answ1 ~ b2 + b3 + b4 + b5 + b7 + b9 + b10 + scene, 
    family = binomial, data = pdata)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-4.2828  -0.0182  -0.0079  -0.0045   3.8098  

Coefficients:
              Estimate Std. Error z value Pr(>|z|)    
(Intercept)   -4.43754   12.15280  -0.365 0.715003    
b2            21.35727   14.89459   1.434 0.151602    
b3          -100.04779   35.09888  -2.850 0.004366 ** 
b4            85.05952   22.54604   3.773 0.000161 ***
b5           -38.14592    3.59761 -10.603  < 2e-16 ***
b7            99.46042    4.05353  24.537  < 2e-16 ***
b9            47.12670    7.24054   6.509 7.58e-11 ***
b10           -0.01923    0.04797  -0.401 0.688521    
sceneS2        4.49335    0.49467   9.084  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 14053.6  on 50538  degrees of freedom
Residual deviance:  1291.7  

Warning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurred

,Df,Deviance,AIC,LRT,Pr(>Chi)
<none>,NA,1291.676,1309.676,NA,NA
b2,1.0000000,1293.7482433,1309.7482433,2.0721370,0.1500112
b3,1.000000e+00,1.299534e+03,1.315534e+03,7.857899e+00,5.059917e-03
b4,1.000000e+00,1.305059e+03,1.321059e+03,1.338269e+01,2.539573e-04
b5,1.000000e+00,1.424819e+03,1.440819e+03,1.331431e+02,8.412265e-31
b7,1.000,5055.419,5071.419,3763.743,0.000
b9,1.000000e+00,1.343497e+03,1.359497e+03,5.182112e+01,6.079422e-13
b10,1.0000000,1291.8368792,1307.8368792,0.1607729,0.6884459
scene,1.000000e+00,1.397027e+03,1.413027e+03,1.053510e+02,1.023088e-24


Выкидываем b10:

In [26]:
simple.glm5 = glm(answ1~ b2 + b3 + b4 + b5 +b7 + b9 + scene, data=pdata, family=binomial)
summary(simple.glm5)
drop1(simple.glm5, test="Chi")

Warning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurred


Call:
glm(formula = answ1 ~ b2 + b3 + b4 + b5 + b7 + b9 + scene, family = binomial, 
    data = pdata)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-4.2833  -0.0181  -0.0078  -0.0044   3.8032  

Coefficients:
            Estimate Std. Error z value Pr(>|z|)    
(Intercept)  -9.2983     0.8449 -11.005  < 2e-16 ***
b2           21.1895    14.8760   1.424 0.154329    
b3          -99.2954    35.0346  -2.834 0.004594 ** 
b4           84.8760    22.5369   3.766 0.000166 ***
b5          -38.4694     3.5044 -10.977  < 2e-16 ***
b7           99.6230     4.0391  24.665  < 2e-16 ***
b9           48.4723     6.4534   7.511 5.86e-14 ***
sceneS2       4.3921     0.4248  10.340  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 14053.6  on 50538  degrees of freedom
Residual deviance:  1291.8  on 50531  degrees of freedom
AIC: 1307.8

Number of Fisher Scoring iterations: 

Warning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurred

,Df,Deviance,AIC,LRT,Pr(>Chi)
<none>,NA,1291.837,1307.837,NA,NA
b2,1.0000000,1293.8817588,1307.8817588,2.0448796,0.1527191
b3,1.000000e+00,1.299609e+03,1.313609e+03,7.772014e+00,5.306184e-03
b4,1.000000e+00,1.305183e+03,1.319183e+03,1.334640e+01,2.589188e-04
b5,1.000000e+00,1.440487e+03,1.454487e+03,1.486501e+02,3.420042e-34
b7,1.000,5110.063,5124.063,3818.226,0.000
b9,1.000000e+00,1.362258e+03,1.376258e+03,7.042144e+01,4.789659e-17
scene,1.000000e+00,1.448253e+03,1.462253e+03,1.564162e+02,6.866208e-36


Избавляемся от незначимого коэффициента b2:

In [27]:
simple.glm6 = glm(answ1~ b3 + b4 + b5 +b7 + b9 + scene, data=pdata, family=binomial)
summary(simple.glm6)
drop1(simple.glm6, test="Chi")

Warning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurred


Call:
glm(formula = answ1 ~ b3 + b4 + b5 + b7 + b9 + scene, family = binomial, 
    data = pdata)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-4.2517  -0.0178  -0.0077  -0.0044   3.7779  

Coefficients:
            Estimate Std. Error z value Pr(>|z|)    
(Intercept)  -8.6519     0.7169 -12.069  < 2e-16 ***
b3          -58.9995    21.2553  -2.776 0.005507 ** 
b4           69.9411    20.2974   3.446 0.000569 ***
b5          -39.8046     3.3812 -11.772  < 2e-16 ***
b7           99.1685     4.0099  24.731  < 2e-16 ***
b9           48.9073     6.4829   7.544 4.56e-14 ***
sceneS2       4.4688     0.4258  10.494  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 14053.6  on 50538  degrees of freedom
Residual deviance:  1293.9  on 50532  degrees of freedom
AIC: 1307.9

Number of Fisher Scoring iterations: 11


Warning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurred

,Df,Deviance,AIC,LRT,Pr(>Chi)
<none>,NA,1293.882,1307.882,NA,NA
b3,1.000000e+00,1.301168e+03,1.313168e+03,7.286727e+00,6.946592e-03
b4,1.000000e+00,1.305237e+03,1.317237e+03,1.135539e+01,7.522941e-04
b5,1.000000e+00,1.492950e+03,1.504950e+03,1.990681e+02,3.335693e-45
b7,1.000,5114.362,5126.362,3820.481,0.000
b9,1.000000e+00,1.367413e+03,1.379413e+03,7.353171e+01,9.903246e-18
scene,1.000000e+00,1.462616e+03,1.474616e+03,1.687340e+02,1.398566e-38


Больше удалять нечего. Эта модель -- максимум точности, которой мы можем достичь при анализе. К сожалению, среди значимых коэффициентов осталась переменная scene, что означает, что в модели должен фигурировать идентификатор сцены. Автоматически это означает, что для каждой сцены, по сути, мы должы будем строить свою модель. Альтернативный вариант -- использование смешанных моделей (mixed effects), что даст значительно усложнение всех вычислений.

Итак, надежда на то, что сцена окажется не значимой, и можно построить модель, в которой не фигурирует идентификатор сцены, не оправдалась. Вернемся на шаг назад, вручную удалим все переменные, которые зависят от сцены (см. первый этап анализа), затем повторим процедуру отбора оптимальной модели и посмотрим, насколько ухудшадся показатели такой модели по сравнению с simple.glm6. 

Ниже повторяется вся процедура отбора переменных без дополнительных комментарив:

In [28]:
simple.glm7 = glm(answ1~b1 + b2 + b3 + b4 + b5 + b6 +b7, data=pdata, family=binomial)
summary(simple.glm7)
drop1(simple.glm7, test="Chi")

Warning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurred


Call:
glm(formula = answ1 ~ b1 + b2 + b3 + b4 + b5 + b6 + b7, family = binomial, 
    data = pdata)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-4.3647  -0.0290  -0.0156  -0.0091   4.3224  

Coefficients:
            Estimate Std. Error z value Pr(>|z|)    
(Intercept)  -15.105      1.288 -11.725  < 2e-16 ***
b1           102.756     36.813   2.791  0.00525 ** 
b2           -65.801     40.518  -1.624  0.10437    
b3           -70.817     33.606  -2.107  0.03509 *  
b4            47.993     21.618   2.220  0.02642 *  
b5           -30.658      3.171  -9.670  < 2e-16 ***
b6            34.449     11.312   3.045  0.00232 ** 
b7            80.444     13.096   6.142 8.13e-10 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 14053.6  on 50538  degrees of freedom
Residual deviance:  1438.2  on 50531  degrees of freedom
AIC: 1454.2

Number of Fisher Scoring iterations: 10


Warning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurred

,Df,Deviance,AIC,LRT,Pr(>Chi)
<none>,NA,1438.155,1454.155,NA,NA
b1,1.000000e+00,1.446026e+03,1.460026e+03,7.871087e+00,5.023144e-03
b2,1.000000,1440.805798,1454.805798,2.651043,0.103482
b3,1.000000e+00,1.442571e+03,1.456571e+03,4.416502e+00,3.559292e-02
b4,1.000000e+00,1.443011e+03,1.457011e+03,4.855902e+00,2.755171e-02
b5,1.000000e+00,1.545974e+03,1.559974e+03,1.078194e+02,2.944222e-25
b6,1.000000e+00,1.447446e+03,1.461446e+03,9.291302e+00,2.302445e-03
b7,1.000000e+00,1.477756e+03,1.491756e+03,3.960169e+01,3.114140e-10


In [29]:
simple.glm8 = glm(answ1~b1 + b3 + b4 + b5 + b6 +b7, data=pdata, family=binomial)
summary(simple.glm8)
drop1(simple.glm8, test="Chi")

Warning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurred


Call:
glm(formula = answ1 ~ b1 + b3 + b4 + b5 + b6 + b7, family = binomial, 
    data = pdata)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-4.3814  -0.0291  -0.0158  -0.0093   4.3174  

Coefficients:
            Estimate Std. Error z value Pr(>|z|)    
(Intercept)  -13.943      1.065 -13.093  < 2e-16 ***
b1            47.000     13.733   3.422 0.000621 ***
b3           -63.187     33.583  -1.882 0.059902 .  
b4            29.917     18.754   1.595 0.110650    
b5           -31.536      3.131 -10.073  < 2e-16 ***
b6            38.963     10.989   3.545 0.000392 ***
b7            75.563     12.743   5.930 3.04e-09 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 14053.6  on 50538  degrees of freedom
Residual deviance:  1440.8  on 50532  degrees of freedom
AIC: 1454.8

Number of Fisher Scoring iterations: 10


Warning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurred

,Df,Deviance,AIC,LRT,Pr(>Chi)
<none>,NA,1440.806,1454.806,NA,NA
b1,1.000000e+00,1.457519e+03,1.469519e+03,1.671286e+01,4.348525e-05
b3,1.000000e+00,1.444625e+03,1.456625e+03,3.819134e+00,5.067041e-02
b4,1.0000000,1443.3783405,1455.3783405,2.5725424,0.1087329
b5,1.000000e+00,1.558526e+03,1.570526e+03,1.177206e+02,1.996197e-27
b6,1.000000e+00,1.453345e+03,1.465345e+03,1.253926e+01,3.984911e-04
b7,1.000000e+00,1.477811e+03,1.489811e+03,3.700510e+01,1.178204e-09


In [30]:
simple.glm9 = glm(answ1~b1 + b3 + b5 + b6 +b7, data=pdata, family=binomial)
summary(simple.glm9)
drop1(simple.glm9, test="Chi")

Warning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurred


Call:
glm(formula = answ1 ~ b1 + b3 + b5 + b6 + b7, family = binomial, 
    data = pdata)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-4.3205  -0.0293  -0.0163  -0.0096   4.2957  

Coefficients:
            Estimate Std. Error z value Pr(>|z|)    
(Intercept)  -13.513      1.000 -13.512  < 2e-16 ***
b1            32.036      9.411   3.404 0.000664 ***
b3           -13.358     10.785  -1.239 0.215518    
b5           -32.009      3.074 -10.411  < 2e-16 ***
b6            42.514     10.723   3.965 7.34e-05 ***
b7            70.289     12.227   5.749 8.99e-09 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 14053.6  on 50538  degrees of freedom
Residual deviance:  1443.4  on 50533  degrees of freedom
AIC: 1455.4

Number of Fisher Scoring iterations: 10


Warning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurred

,Df,Deviance,AIC,LRT,Pr(>Chi)
<none>,NA,1443.378,1455.378,NA,NA
b1,1.000000e+00,1.457523e+03,1.467523e+03,1.414423e+01,1.693160e-04
b3,1.0000000,1444.9792568,1454.9792568,1.6009163,0.2057734
b5,1.000000e+00,1.567276e+03,1.577276e+03,1.238977e+02,8.869957e-29
b6,1.000000e+00,1.459115e+03,1.469115e+03,1.573627e+01,7.281458e-05
b7,1.000000e+00,1.477851e+03,1.487851e+03,3.447230e+01,4.323603e-09


In [31]:
simple.glm10 = glm(answ1~b1 + b5 + b6 +b7, data=pdata, family=binomial)
summary(simple.glm10)
drop1(simple.glm10, test="Chi")

Warning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurred


Call:
glm(formula = answ1 ~ b1 + b5 + b6 + b7, family = binomial, data = pdata)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-4.3348  -0.0290  -0.0160  -0.0096   4.2994  

Coefficients:
            Estimate Std. Error z value Pr(>|z|)    
(Intercept) -12.4185     0.4587 -27.073  < 2e-16 ***
b1           20.8339     2.6116   7.977 1.49e-15 ***
b5          -34.7703     2.1604 -16.095  < 2e-16 ***
b6           45.3291    10.4937   4.320 1.56e-05 ***
b7           67.9584    12.0931   5.620 1.91e-08 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 14054  on 50538  degrees of freedom
Residual deviance:  1445  on 50534  degrees of freedom
AIC: 1455

Number of Fisher Scoring iterations: 10


Warning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurred

,Df,Deviance,AIC,LRT,Pr(>Chi)
<none>,NA,1444.979,1454.979,NA,NA
b1,1.000000e+00,1.514534e+03,1.522534e+03,6.955486e+01,7.431837e-17
b5,1.000000e+00,2.008890e+03,2.016890e+03,5.639107e+02,1.185530e-124
b6,1.000000e+00,1.463680e+03,1.471680e+03,1.870055e+01,1.529382e-05
b7,1.000000e+00,1.477961e+03,1.485961e+03,3.298191e+01,9.302018e-09


Итак, в этом случае оптимальный набор переменных: b1, b5, b6, b7. AIC: 1455 (для сравнения: AIC у simple.glm6 равен 1307.9).

<a id='CloudsValidation'></a>
##### Валидация модели

Посмотрим, нет ли каких-либо закономерностей в ответах. Для этого построим график зависимостей остатков от подогнанных значений.

Для оценки качества модели изучим поведение зависимости остатков (невязка модели) от прогнозируемой величины и от независимых переменных. В идеале остатки не должны показывать каких-либо закономерностей.

In [32]:
E10 = resid(simple.glm10, type="response")
png("Img/CMASK/glm10_res_fitted.png", width=728, height=728, units="px")
  plot(simple.glm10$fitted.values, E10, cex=0.1)
dev.off()

png 
  2

<img src="https://176.9.38.120/cruncher/files/deforestation/Img/CMASK/glm10_res_fitted.png">

Поскольку ответ логистической регрессии это один или ноль, а значений, использующихся для подгонки достаточно много, то на графике видим две полосы точек (остатков). В итогое понять, есть ли закономерность в остатках становится сложно. Поэтому упорядочим полученные ответы по возрастанию fitted.values, разобьем их на группы и посчитаем в каждой группе среднее значение невязки (средний остаток).

In [33]:
ER10 = data.frame(err=E10, fitted=simple.glm10$fitted.values, 
                b1=pdata$b1, b2=pdata$b2, b3=pdata$b3, b4=pdata$b4, b5=pdata$b5, b6=pdata$b6, 
                b7=pdata$b7, b9=pdata$b9, b10=pdata$b10, b11=pdata$b11, scene=pdata$scene)

# Сортируем по порядку прогнозируемых величин
ER10 = ER10[order(ER10$fitted), ]

# Разбиваем на группы
group_size = 100
ER10 = cbind(id=(1:length(ER10$fitted))%/% group_size, ER10)
# Вычислим среднее значение ошибки в каждой группе и поместим в таблицу
lookup_table = aggregate(ER10['err'], by=list(ER10$id), FUN=mean)
names(lookup_table) = c('id', 'err')

# Создадим вектор из средний значений ошибок 
a = sapply(ER10$id, function(id){lookup_table[lookup_table$id==id, 'err']})

ER10 = cbind(ER10, mean.err=a)


png("Img/CMASK/glm10_res_fitted_srt.png", width=728, height=728, units="px")
  plot(ER10$fitted, ER10$mean.err, cex=0.1)
dev.off()

png 
  2

<img src="https://176.9.38.120/cruncher/files/deforestation/Img/CMASK/glm10_res_fitted_srt.png">

Как видим, в усредненных остатках есть некоторая тенденция: если прогнозируемое значение близко к нулю (не облако), то остаток скорее будет отцательным, для средних прогнозируемых значений остатки чаще положительны, а если прогнозируемое значение близко к единице, то отстаток опять будет скорее отрицательным. Для идеально подогнанной кривой усредненные остатки должны быть более-менее равномерно разбросаны вокруг нуля и не зависеть от прогнозируемой величины.

Аналогично, построим графики зависимостей остатков от всех переменных (как использованных в модели, так и не использованных).

In [34]:
png("Img/CMASK/glm10_res_bXX_srt.png", width=728, height=728, units="px")
    par(mfrow = c(3, 4))
    plot(ER10$b1, ER10$mean.err, cex=0.1)
    plot(ER10$b2, ER10$mean.err, cex=0.1)
    plot(ER10$b3, ER10$mean.err, cex=0.1)
    plot(ER10$b4, ER10$mean.err, cex=0.1)

    plot(ER10$b5, ER10$mean.err, cex=0.1)
    plot(ER10$b6, ER10$mean.err, cex=0.1)
    plot(ER10$b7, ER10$mean.err, cex=0.1)

    plot(ER10$b9, ER10$mean.err, cex=0.1)

    plot(ER10$b10, ER10$mean.err, cex=0.1)
    plot(ER10$b11, ER10$mean.err, cex=0.1)

    boxplot(ER10$mean.err ~ ER10$scene, cex=0.1)

    par(opar)
dev.off()

png 
  2

<img src="https://176.9.38.120/cruncher/files/deforestation/Img/CMASK/glm10_res_bXX_srt.png">

Каких-то бросающихся в глаза тенденций (например, низких значений остатков при высоких величинах переменных или чего-нибудь в этом роде) в зависимости остатков от каналов не заметно: отрицательные остатки обычно уравновешиваются положительными. Также хорошая новость в том, что остатки не показывают зависимости от сцены.

<a id='CloudsResume'></a>
##### Вывод

Таким образом возьмем в качестве первого приближения данную модель:

In [36]:
cloud.model = simple.glm10

Будем считать с некоторыми оговорками (см. закономерности в остатах на графике остатки-прогнозные величины), что модель отражает реальное положение дел.

<a id='Snow'></a>
#### Снег-не-снег

<a id='SnowModel1'></a>
#### Построение модели

Построим модель определяющую, явлеяется ли данный пиксель снегом. Для начала построим модель со всеми имеющимися переменными, включая название сцены. Логика построения следует логике, описанной в разделе построения модели по облакам. Поэтому подробности здесь опускаются.

In [38]:
simple.glm11 = glm(answ2~b1 + b2 + b3 + b4 + b5 + b6 +b7 + b9 + b10 + b11 + scene, data=pdata, family=binomial)
summary(simple.glm11)
drop1(simple.glm11, test="Chi")


Call:
glm(formula = answ2 ~ b1 + b2 + b3 + b4 + b5 + b6 + b7 + b9 + 
    b10 + b11 + scene, family = binomial, data = pdata)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-5.4517  -0.0508  -0.0264  -0.0154   4.6237  

Coefficients:
             Estimate Std. Error z value Pr(>|z|)    
(Intercept)  33.24812    5.69266   5.841 5.20e-09 ***
b1          -14.17302   10.64750  -1.331  0.18315    
b2           10.32504   11.31638   0.912  0.36156    
b3          -23.81504    9.11468  -2.613  0.00898 ** 
b4           71.49703    6.89834  10.364  < 2e-16 ***
b5           -4.11483    1.46946  -2.800  0.00511 ** 
b6           34.39465    8.34044   4.124 3.73e-05 ***
b7          -85.42956    9.90302  -8.627  < 2e-16 ***
b9            1.81237    2.95947   0.612  0.54028    
b10           0.03536    0.07452   0.474  0.63516    
b11          -0.20574    0.07156  -2.875  0.00404 ** 
sceneS2      -3.08600    0.22752 -13.564  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01

Warning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurred

,Df,Deviance,AIC,LRT,Pr(>Chi)
<none>,NA,4111.006,4135.006,NA,NA
b1,1.0000000,4112.7820595,4134.7820595,1.7756547,0.1826838
b2,1.0000000,4111.8393444,4133.8393444,0.8329396,0.3614239
b3,1.000000e+00,4.117866e+03,4.139866e+03,6.859756e+00,8.815857e-03
b4,1.000000e+00,4.222330e+03,4.244330e+03,1.113240e+02,5.024817e-26
b5,1.000000e+00,4.118951e+03,4.140951e+03,7.945046e+00,4.821919e-03
b6,1.000000e+00,4.128463e+03,4.150463e+03,1.745688e+01,2.938990e-05
b7,1.000000e+00,4.189994e+03,4.211994e+03,7.898715e+01,6.251515e-19
b9,1.0000000,4111.3809981,4133.3809981,0.3745933,0.5405111
b10,1.0000000,4111.2316338,4133.2316338,0.2252290,0.6350842


In [ ]:
drop1(simple.glm11, test="Chi")

In [39]:
simple.glm12 = glm(answ2~b1 + b2 + b3 + b4 + b5 + b6 +b7 + b9 + b11 + scene, data=pdata, family=binomial)
summary(simple.glm12)
drop1(simple.glm12, test="Chi")


Call:
glm(formula = answ2 ~ b1 + b2 + b3 + b4 + b5 + b6 + b7 + b9 + 
    b11 + scene, family = binomial, data = pdata)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-5.4573  -0.0508  -0.0263  -0.0154   4.6179  

Coefficients:
             Estimate Std. Error z value Pr(>|z|)    
(Intercept)  34.09838    5.40337   6.311 2.78e-10 ***
b1          -14.72704   10.58590  -1.391  0.16417    
b2           10.48689   11.31599   0.927  0.35407    
b3          -23.17515    9.01273  -2.571  0.01013 *  
b4           71.15259    6.86105  10.371  < 2e-16 ***
b5           -4.07003    1.46638  -2.776  0.00551 ** 
b6           34.46864    8.31974   4.143 3.43e-05 ***
b7          -85.55749    9.87576  -8.663  < 2e-16 ***
b9            2.36791    2.71726   0.871  0.38352    
b11          -0.17332    0.02123  -8.166 3.20e-16 ***
sceneS2      -3.06232    0.22178 -13.808  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial 

Warning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurred

,Df,Deviance,AIC,LRT,Pr(>Chi)
<none>,NA,4111.232,4133.232,NA,NA
b1,1.0000000,4113.1714514,4133.1714514,1.9398176,0.1636883
b2,1.0000000,4112.0909800,4132.0909800,0.8593462,0.3539217
b3,1.000000e+00,4.117874e+03,4.137874e+03,6.642334e+00,9.958336e-03
b4,1.000000e+00,4.222863e+03,4.242863e+03,1.116313e+02,4.303379e-26
b5,1.000000e+00,4.119037e+03,4.139037e+03,7.805142e+00,5.209777e-03
b6,1.000000e+00,4.128843e+03,4.148843e+03,1.761181e+01,2.709011e-05
b7,1.000000e+00,4.190758e+03,4.210758e+03,7.952612e+01,4.758928e-19
b9,1.0000000,4111.9898219,4131.9898219,0.7581881,0.3838962
b11,1.000000e+00,4.179633e+03,4.199633e+03,6.840134e+01,1.333869e-16


In [40]:
simple.glm13 = glm(answ2~b1 + b2 + b3 + b4 + b5 + b6 +b7 + b11 + scene, data=pdata, family=binomial)
summary(simple.glm13)
drop1(simple.glm13, test="Chi")


Call:
glm(formula = answ2 ~ b1 + b2 + b3 + b4 + b5 + b6 + b7 + b11 + 
    scene, family = binomial, data = pdata)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-5.4431  -0.0509  -0.0263  -0.0152   4.6231  

Coefficients:
             Estimate Std. Error z value Pr(>|z|)    
(Intercept)  36.81251    4.43940   8.292  < 2e-16 ***
b1          -15.17628   10.57761  -1.435  0.15136    
b2           10.79602   11.31651   0.954  0.34008    
b3          -22.05583    8.91525  -2.474  0.01336 *  
b4           69.92144    6.70580  10.427  < 2e-16 ***
b5           -3.79797    1.43135  -2.653  0.00797 ** 
b6           34.88959    8.36651   4.170 3.04e-05 ***
b7          -85.25847    9.94872  -8.570  < 2e-16 ***
b11          -0.18434    0.01715 -10.749  < 2e-16 ***
sceneS2      -3.02956    0.21920 -13.821  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 26184  on 50538  

Warning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurred

,Df,Deviance,AIC,LRT,Pr(>Chi)
<none>,NA,4111.99,4131.99,NA,NA
b1,1.0000000,4114.0531486,4132.0531486,2.0633267,0.1508805
b2,1.0000000,4112.9005246,4130.9005246,0.9107027,0.3399281
b3,1.000000e+00,4.118135e+03,4.136135e+03,6.145505e+00,1.317465e-02
b4,1.000000e+00,4.224519e+03,4.242519e+03,1.125294e+02,2.735712e-26
b5,1.000000e+00,4.119116e+03,4.137116e+03,7.126318e+00,7.596057e-03
b6,1.000000e+00,4.129763e+03,4.147763e+03,1.777360e+01,2.488116e-05
b7,1.000000e+00,4.190824e+03,4.208824e+03,7.883422e+01,6.754667e-19
b11,1.000000e+00,4.221619e+03,4.239619e+03,1.096292e+02,1.181485e-25
scene,1.000000e+00,4.322975e+03,4.340975e+03,2.109850e+02,8.374492e-48


In [41]:
simple.glm14 = glm(answ2~b1 + b3 + b4 + b5 + b6 +b7 + b11 + scene, data=pdata, family=binomial)
summary(simple.glm14)
drop1(simple.glm14, test="Chi")


Call:
glm(formula = answ2 ~ b1 + b3 + b4 + b5 + b6 + b7 + b11 + scene, 
    family = binomial, data = pdata)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-5.4438  -0.0509  -0.0262  -0.0152   4.6187  

Coefficients:
             Estimate Std. Error z value Pr(>|z|)    
(Intercept)  36.17094    4.39281   8.234  < 2e-16 ***
b1           -5.78434    3.85134  -1.502  0.13312    
b3          -23.09351    8.84408  -2.611  0.00902 ** 
b4           72.35617    6.20721  11.657  < 2e-16 ***
b5           -3.48863    1.39372  -2.503  0.01231 *  
b6           33.55301    8.24377   4.070  4.7e-05 ***
b7          -83.75310    9.81167  -8.536  < 2e-16 ***
b11          -0.18271    0.01708 -10.694  < 2e-16 ***
sceneS2      -3.05746    0.21716 -14.079  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 26183.7  on 50538  degrees of freedom
Residual deviance:  4112.9  on 50530  

Warning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurred

,Df,Deviance,AIC,LRT,Pr(>Chi)
<none>,NA,4112.901,4130.901,NA,NA
b1,1.0000000,4115.1544676,4131.1544676,2.2539431,0.1332744
b3,1.000000e+00,4.119748e+03,4.135748e+03,6.847703e+00,8.875530e-03
b4,1.000000e+00,4.254200e+03,4.270200e+03,1.412991e+02,1.384008e-32
b5,1.0000000,4119.2409890,4135.2409890,6.3404644,0.0118014
b6,1.000000e+00,4.129821e+03,4.145821e+03,1.692082e+01,3.897174e-05
b7,1.000000e+00,4.191100e+03,4.207100e+03,7.819986e+01,9.312503e-19
b11,1.000000e+00,4.221650e+03,4.237650e+03,1.087494e+02,1.841552e-25
scene,1.000000e+00,4.333768e+03,4.349768e+03,2.208676e+02,5.849799e-50


In [42]:
simple.glm15 = glm(answ2 ~ b3 + b4 + b5 + b6 +b7 + b11 + scene, data=pdata, family=binomial)
summary(simple.glm15)
drop1(simple.glm15, test="Chi")


Call:
glm(formula = answ2 ~ b3 + b4 + b5 + b6 + b7 + b11 + scene, family = binomial, 
    data = pdata)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-5.4249  -0.0506  -0.0259  -0.0150   4.6135  

Coefficients:
             Estimate Std. Error z value Pr(>|z|)    
(Intercept)  34.01617    4.14562   8.205 2.30e-16 ***
b3          -32.34756    6.36748  -5.080 3.77e-07 ***
b4           74.41082    6.05557  12.288  < 2e-16 ***
b5           -2.15573    1.06873  -2.017 0.043685 *  
b6           30.29606    7.90794   3.831 0.000128 ***
b7          -80.01316    9.42609  -8.488  < 2e-16 ***
b11          -0.17615    0.01649 -10.679  < 2e-16 ***
sceneS2      -3.14354    0.20940 -15.012  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 26183.7  on 50538  degrees of freedom
Residual deviance:  4115.2  on 50531  degrees of freedom
AIC: 4131.2

Number of Fisher Scoring it

Warning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurred

,Df,Deviance,AIC,LRT,Pr(>Chi)
<none>,NA,4115.154,4131.154,NA,NA
b3,1.000000e+00,4.140997e+03,4.154997e+03,2.584229e+01,3.704807e-07
b4,1.000000e+00,4.273135e+03,4.287135e+03,1.579808e+02,3.124854e-36
b5,1.000000e+00,4.119261e+03,4.133261e+03,4.106518e+00,4.271824e-02
b6,1.000000e+00,4.130038e+03,4.144038e+03,1.488334e+01,1.143688e-04
b7,1.000000e+00,4.191505e+03,4.205505e+03,7.635068e+01,2.375096e-18
b11,1.000000e+00,4.222762e+03,4.236762e+03,1.076074e+02,3.276637e-25
scene,1.000000e+00,4.373083e+03,4.387083e+03,2.579285e+02,4.853573e-58


In [43]:
simple.glm16 = glm(answ2 ~ b3 + b4 + b6 +b7 + b11 + scene, data=pdata, family=binomial)
summary(simple.glm16)
drop1(simple.glm16, test="Chi")


Call:
glm(formula = answ2 ~ b3 + b4 + b6 + b7 + b11 + scene, family = binomial, 
    data = pdata)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-5.3954  -0.0504  -0.0255  -0.0148   4.5795  

Coefficients:
             Estimate Std. Error z value Pr(>|z|)    
(Intercept)  34.19077    4.12776   8.283  < 2e-16 ***
b3          -25.06529    5.25553  -4.769 1.85e-06 ***
b4           66.09273    4.42352  14.941  < 2e-16 ***
b6           28.22973    7.78989   3.624  0.00029 ***
b7          -78.21511    9.32161  -8.391  < 2e-16 ***
b11          -0.17823    0.01639 -10.873  < 2e-16 ***
sceneS2      -3.10642    0.20840 -14.906  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 26183.7  on 50538  degrees of freedom
Residual deviance:  4119.3  on 50532  degrees of freedom
AIC: 4133.3

Number of Fisher Scoring iterations: 9


Warning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurred

,Df,Deviance,AIC,LRT,Pr(>Chi)
<none>,NA,4119.261,4133.261,NA,NA
b3,1.000000e+00,4.142048e+03,4.154048e+03,2.278739e+01,1.809492e-06
b4,1.000000e+00,4.355058e+03,4.367058e+03,2.357970e+02,3.244709e-53
b6,1.000000e+00,4.132546e+03,4.144546e+03,1.328467e+01,2.675846e-04
b7,1.000000e+00,4.193610e+03,4.205610e+03,7.434896e+01,6.545960e-18
b11,1.000000e+00,4.231264e+03,4.243264e+03,1.120034e+02,3.566982e-26
scene,1.000000e+00,4.373469e+03,4.385469e+03,2.542077e+02,3.141538e-57


Здесь также, как и в предыдущем случае, остались переменные, зависящие от сцены. У данной модели AIC равен 4133.3. Построим модель, в которой не будет тепловых каналов и идентификатора сцены.

In [48]:
simple.glm17 = glm(answ2 ~ b1 + b2 + b3 + b4 + b5 + b6 +b7, data=pdata, family=binomial)
summary(simple.glm17)
drop1(simple.glm17, test="Chi")


Call:
glm(formula = answ2 ~ b1 + b2 + b3 + b4 + b5 + b6 + b7, family = binomial, 
    data = pdata)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-5.6076  -0.0759  -0.0432  -0.0320   3.6605  

Coefficients:
            Estimate Std. Error z value Pr(>|z|)    
(Intercept)   -4.730      0.426 -11.103  < 2e-16 ***
b1           -57.366      9.232  -6.213 5.18e-10 ***
b2            46.454     10.064   4.616 3.91e-06 ***
b3           -23.972      8.120  -2.952  0.00316 ** 
b4            67.635      5.946  11.375  < 2e-16 ***
b5            -7.566      1.342  -5.638 1.72e-08 ***
b6           -12.266      7.752  -1.582  0.11357    
b7           -25.834      9.313  -2.774  0.00554 ** 
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 26183.7  on 50538  degrees of freedom
Residual deviance:  5135.5  on 50531  degrees of freedom
AIC: 5151.5

Number of Fisher Scoring iterations: 8


Warning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurred

,Df,Deviance,AIC,LRT,Pr(>Chi)
<none>,NA,5135.475,5151.475,NA,NA
b1,1.000000e+00,5.174659e+03,5.188659e+03,3.918435e+01,3.856181e-10
b2,1.000000e+00,5.156883e+03,5.170883e+03,2.140826e+01,3.711685e-06
b3,1.000000e+00,5.144243e+03,5.158243e+03,8.768644e+00,3.064533e-03
b4,1.000000e+00,5.270597e+03,5.284597e+03,1.351221e+02,3.104808e-31
b5,1.000000e+00,5.168265e+03,5.182265e+03,3.278981e+01,1.026816e-08
b6,1.0000000,5137.9712192,5151.9712192,2.4965271,0.1140977
b7,1.000000e+00,5.143285e+03,5.157285e+03,7.809859e+00,5.196196e-03


In [49]:
simple.glm18 = glm(answ2 ~ b1 + b2 + b3 + b4 + b5 +b7, data=pdata, family=binomial)
summary(simple.glm18)
drop1(simple.glm18, test="Chi")


Call:
glm(formula = answ2 ~ b1 + b2 + b3 + b4 + b5 + b7, family = binomial, 
    data = pdata)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-5.6318  -0.0763  -0.0435  -0.0324   3.6541  

Coefficients:
            Estimate Std. Error z value Pr(>|z|)    
(Intercept)  -4.6623     0.4245 -10.984  < 2e-16 ***
b1          -60.6583     9.0127  -6.730 1.69e-11 ***
b2           48.9352     9.9530   4.917 8.81e-07 ***
b3          -21.1969     7.9413  -2.669   0.0076 ** 
b4           66.8803     5.9383  11.263  < 2e-16 ***
b5           -8.1754     1.2890  -6.342 2.26e-10 ***
b7          -40.4613     1.2929 -31.296  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 26184  on 50538  degrees of freedom
Residual deviance:  5138  on 50532  degrees of freedom
AIC: 5152

Number of Fisher Scoring iterations: 8


Warning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurred

,Df,Deviance,AIC,LRT,Pr(>Chi)
<none>,NA,5137.971,5151.971,NA,NA
b1,1.000000e+00,5.183923e+03,5.195923e+03,4.595177e+01,1.212001e-11
b2,1.000000e+00,5.162247e+03,5.174247e+03,2.427528e+01,8.350384e-07
b3,1.000000e+00,5.145148e+03,5.157148e+03,7.176870e+00,7.384942e-03
b4,1.000000e+00,5.270858e+03,5.282858e+03,1.328873e+02,9.569505e-31
b5,1.000000e+00,5.179596e+03,5.191596e+03,4.162431e+01,1.106093e-10
b7,1.000,7193.120,7205.120,2055.149,0.000


Итак оптимальный набор переменных: b1, b2, b3, b4, b5, b7. AIC: 5152 (для сравнения: AIC у simple.glm16 равен 4133.3).

<a id='SnowValidation1'></a>
##### Валидация модели

In [51]:
E18 = resid(simple.glm18, type="response")
ER18 = data.frame(err=E18, fitted=simple.glm18$fitted.values, 
                b1=pdata$b1, b2=pdata$b2, b3=pdata$b3, b4=pdata$b4, b5=pdata$b5, b6=pdata$b6, 
                b7=pdata$b7, b9=pdata$b9, b10=pdata$b10, b11=pdata$b11, scene=pdata$scene)

# Сортируем по порядку прогнозируемых величин
ER18 = ER18[order(ER18$fitted), ]

# Разбиваем на группы
group_size = 100
ER18 = cbind(id=(1:length(ER18$fitted))%/% group_size, ER18)
# Вычислим среднее значение ошибки в каждой группе и поместим в таблицу
lookup_table = aggregate(ER18['err'], by=list(ER18$id), FUN=mean)
names(lookup_table) = c('id', 'err')

# Создадим вектор из средний значений ошибок 
a = sapply(ER18$id, function(id){lookup_table[lookup_table$id==id, 'err']})

ER18 = cbind(ER18, mean.err=a)


png("Img/CMASK/glm18_res_fitted_srt.png", width=728, height=728, units="px")
  plot(ER18$fitted, ER18$mean.err, cex=0.1)
dev.off()

png 
  2

<img src="https://176.9.38.120/cruncher/files/deforestation/Img/CMASK/glm18_res_fitted_srt.png">

Как и ранее, в остатках очень нехорошее поведение -- для средних значений прогнозных величин остатки имеют тенденцию увеличиваться.

Зависимость остатков от объясняющих переменных:

In [52]:
png("Img/CMASK/glm18_res_bXX_srt.png", width=728, height=728, units="px")
    par(mfrow = c(3, 4))
    plot(ER18$b1, ER18$mean.err, cex=0.1)
    plot(ER18$b2, ER18$mean.err, cex=0.1)
    plot(ER18$b3, ER18$mean.err, cex=0.1)
    plot(ER18$b4, ER18$mean.err, cex=0.1)

    plot(ER18$b5, ER18$mean.err, cex=0.1)
    plot(ER18$b6, ER18$mean.err, cex=0.1)
    plot(ER18$b7, ER18$mean.err, cex=0.1)

    plot(ER18$b9, ER18$mean.err, cex=0.1)

    plot(ER18$b10, ER18$mean.err, cex=0.1)
    plot(ER18$b11, ER18$mean.err, cex=0.1)

    boxplot(ER18$mean.err ~ ER18$scene, cex=0.1)

    par(opar)
dev.off()

png 
  2

<img src="https://176.9.38.120/cruncher/files/deforestation/Img/CMASK/glm18_res_bXX_srt.png">

В целом графики остатков чуть хуже, чем для модели облачности. В частности, видно, что остатки для второй сцены имеют больший разброс, чем для первой, заметны также некоторые закономерности в размещении остатков в зависимости от значений объясняющих переменных. Например, для значений b5 из диапазона 0.6--0.8 остатки с большой вероятностью будут отрицательными. Аналогичные проблемы видны в b3, b4 и b9, но 9-й канал у нас не используется в модели, поскольку температура очень сильно колеблется от сцены к сцене.

<a id='SnowModel2'></a>
#### Построение модели-2

Поскольку модель, построенная в предыдущем разделе показывает не очень хорошее поведение остатков в зависимости от объясняющих переменных, попробуем ее улучшить. Для этого добавим в модель квадратичные члены для b3, b4, b5 и посмотрим, улучшится ли модель.

In [53]:
simple.glm19 = glm(answ2 ~ b1 + b2 + b3 + b4 + b5 + b6 +b7 + I(b3^2) + I(b4^2) + I(b5^2), data=pdata, family=binomial)
summary(simple.glm19)
drop1(simple.glm19, test="Chi")


Call:
glm(formula = answ2 ~ b1 + b2 + b3 + b4 + b5 + b6 + b7 + I(b3^2) + 
    I(b4^2) + I(b5^2), family = binomial, data = pdata)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-4.3219  -0.0398  -0.0139  -0.0065   4.5141  

Coefficients:
             Estimate Std. Error z value Pr(>|z|)    
(Intercept)   -8.1972     0.6243 -13.130  < 2e-16 ***
b1           -94.5588     8.8780 -10.651  < 2e-16 ***
b2            76.7963     9.9764   7.698 1.38e-14 ***
b3             7.1260    11.9593   0.596   0.5513    
b4           118.2437    10.6754  11.076  < 2e-16 ***
b5           -48.2176     4.7824 -10.082  < 2e-16 ***
b6             6.2635     7.2951   0.859   0.3906    
b7           -38.3978     8.6063  -4.462 8.13e-06 ***
I(b3^2)       19.9117     8.5997   2.315   0.0206 *  
I(b4^2)     -109.8865     8.7341 -12.581  < 2e-16 ***
I(b5^2)       39.3102     4.8756   8.063 7.47e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter fo

,Df,Deviance,AIC,LRT,Pr(>Chi)
<none>,NA,4383.646,4405.646,NA,NA
b1,1.000000e+00,4.501077e+03,4.521077e+03,1.174315e+02,2.309395e-27
b2,1.000000e+00,4.443611e+03,4.463611e+03,5.996528e+01,9.654562e-15
b3,1.0000000,4384.0004338,4404.0004338,0.3549280,0.5513367
b4,1.000000e+00,4.507220e+03,4.527220e+03,1.235748e+02,1.043764e-28
b5,1.000000e+00,4.497308e+03,4.517308e+03,1.136623e+02,1.545024e-26
b6,1.0000000,4384.3836678,4404.3836678,0.7381619,0.3902503
b7,1.000000e+00,4.404078e+03,4.424078e+03,2.043274e+01,6.176414e-06
I(b3^2),1.000000e+00,4.388833e+03,4.408833e+03,5.187063e+00,2.275564e-02
I(b4^2),1.000000e+00,4.521703e+03,4.541703e+03,1.380578e+02,7.078595e-32


In [54]:
simple.glm20 = glm(answ2 ~ b1 + b2 + b4 + b5 + b6 +b7 + I(b3^2) + I(b4^2) + I(b5^2), data=pdata, family=binomial)
summary(simple.glm20)
drop1(simple.glm20, test="Chi")


Call:
glm(formula = answ2 ~ b1 + b2 + b4 + b5 + b6 + b7 + I(b3^2) + 
    I(b4^2) + I(b5^2), family = binomial, data = pdata)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-4.3308  -0.0398  -0.0140  -0.0065   4.5190  

Coefficients:
             Estimate Std. Error z value Pr(>|z|)    
(Intercept)   -8.0645     0.5821 -13.854  < 2e-16 ***
b1           -92.7050     8.3211 -11.141  < 2e-16 ***
b2            75.8724     9.8649   7.691 1.46e-14 ***
b4           123.5774     5.8349  21.179  < 2e-16 ***
b5           -48.7298     4.7049 -10.357  < 2e-16 ***
b6             5.6561     7.2329   0.782    0.434    
b7           -37.7132     8.5405  -4.416 1.01e-05 ***
I(b3^2)       24.1449     4.8031   5.027 4.98e-07 ***
I(b4^2)     -113.1112     6.8502 -16.512  < 2e-16 ***
I(b5^2)       39.8459     4.7905   8.318  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 26184 

,Df,Deviance,AIC,LRT,Pr(>Chi)
<none>,NA,4384,4404,NA,NA
b1,1.000000e+00,4.514639e+03,4.532639e+03,1.306385e+02,2.970643e-30
b2,1.000000e+00,4.444088e+03,4.462088e+03,6.008740e+01,9.073754e-15
b4,1.000000e+00,4.960808e+03,4.978808e+03,5.768075e+02,1.855752e-127
b5,1.000000e+00,4.505077e+03,4.523077e+03,1.210762e+02,3.677245e-28
b6,1.0000000,4384.6130013,4402.6130013,0.6125675,0.4338228
b7,1.000000e+00,4.404083e+03,4.422083e+03,2.008251e+01,7.417181e-06
I(b3^2),1.000000e+00,4.409985e+03,4.427985e+03,2.598474e+01,3.441267e-07
I(b4^2),1.000000e+00,4.646765e+03,4.664765e+03,2.627642e+02,4.285389e-59
I(b5^2),1.000000e+00,4.461275e+03,4.479275e+03,7.727437e+01,1.487845e-18


In [55]:
simple.glm21 = glm(answ2 ~ b1 + b2 + b4 + b5 +b7 + I(b3^2) + I(b4^2) + I(b5^2), data=pdata, family=binomial)
summary(simple.glm21)
drop1(simple.glm21, test="Chi")


Call:
glm(formula = answ2 ~ b1 + b2 + b4 + b5 + b7 + I(b3^2) + I(b4^2) + 
    I(b5^2), family = binomial, data = pdata)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-4.3185  -0.0398  -0.0140  -0.0065   4.5174  

Coefficients:
            Estimate Std. Error z value Pr(>|z|)    
(Intercept)   -8.091      0.581 -13.927  < 2e-16 ***
b1           -91.610      8.187 -11.190  < 2e-16 ***
b2            74.884      9.772   7.663 1.81e-14 ***
b4           122.713      5.717  21.464  < 2e-16 ***
b5           -48.133      4.638 -10.379  < 2e-16 ***
b7           -31.108      1.167 -26.648  < 2e-16 ***
I(b3^2)       23.435      4.687   5.000 5.74e-07 ***
I(b4^2)     -112.092      6.706 -16.716  < 2e-16 ***
I(b5^2)       39.501      4.769   8.283  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 26183.7  on 50538  degrees of freedom
Residual deviance:  4384.6  on 50530 

,Df,Deviance,AIC,LRT,Pr(>Chi)
<none>,NA,4384.613,4402.613,NA,NA
b1,1.000000e+00,4.515572e+03,4.531572e+03,1.309591e+02,2.527597e-30
b2,1.000000e+00,4.444103e+03,4.460103e+03,5.948960e+01,1.229423e-14
b4,1.000000e+00,4.975129e+03,4.991129e+03,5.905160e+02,1.934956e-130
b5,1.000000e+00,4.506136e+03,4.522136e+03,1.215230e+02,2.935795e-28
b7,1.000,6054.201,6070.201,1669.588,0.000
I(b3^2),1.000000e+00,4.409987e+03,4.425987e+03,2.537362e+01,4.723338e-07
I(b4^2),1.000000e+00,4.648852e+03,4.664852e+03,2.642395e+02,2.043795e-59
I(b5^2),1.000000e+00,4.461277e+03,4.477277e+03,7.666359e+01,2.027064e-18


Итак новый оптимальный набор переменных: b1, b2, b3, b4, b5, b7 и квадратичные члены каналов b3, b4, b5. AIC модели значительно улучшился, дойдя до значения 4402.6 (для сравнения: AIC у simple.glm16 равен 4133.3).

<a id='SnowValidation2'></a>
##### Валидация модели-2

In [56]:
E21 = resid(simple.glm21, type="response")
ER21 = data.frame(err=E21, fitted=simple.glm21$fitted.values, 
                b1=pdata$b1, b2=pdata$b2, b3=pdata$b3, b4=pdata$b4, b5=pdata$b5, b6=pdata$b6, 
                b7=pdata$b7, b9=pdata$b9, b10=pdata$b10, b11=pdata$b11, scene=pdata$scene)

# Сортируем по порядку прогнозируемых величин
ER21 = ER21[order(ER21$fitted), ]

# Разбиваем на группы
group_size = 100
ER21 = cbind(id=(1:length(ER21$fitted))%/% group_size, ER21)
# Вычислим среднее значение ошибки в каждой группе и поместим в таблицу
lookup_table = aggregate(ER21['err'], by=list(ER21$id), FUN=mean)
names(lookup_table) = c('id', 'err')

# Создадим вектор из средний значений ошибок 
a = sapply(ER21$id, function(id){lookup_table[lookup_table$id==id, 'err']})

ER21 = cbind(ER21, mean.err=a)


png("Img/CMASK/glm21_res_fitted_srt.png", width=728, height=728, units="px")
  plot(ER21$fitted, ER21$mean.err, cex=0.1)
dev.off()

png 
  2

<img src="https://176.9.38.120/cruncher/files/deforestation/Img/CMASK/glm21_res_fitted_srt.png">

Проблемы подгонки модели сохранились: остатки для средних значений прогнозируемой величины положительны, а для низких и высоких величин -- отрицательны.

Посмотрим на зависимость остатков от объясняющих переменных.

In [57]:
png("Img/CMASK/glm21_res_bXX_srt.png", width=728, height=728, units="px")
    par(mfrow = c(3, 4))
    plot(ER21$b1, ER21$mean.err, cex=0.1)
    plot(ER21$b2, ER21$mean.err, cex=0.1)
    plot(ER21$b3, ER21$mean.err, cex=0.1)
    plot(ER21$b4, ER21$mean.err, cex=0.1)

    plot(ER21$b5, ER21$mean.err, cex=0.1)
    plot(ER21$b6, ER21$mean.err, cex=0.1)
    plot(ER21$b7, ER21$mean.err, cex=0.1)

    plot(ER21$b9, ER21$mean.err, cex=0.1)

    plot(ER21$b10, ER21$mean.err, cex=0.1)
    plot(ER21$b11, ER21$mean.err, cex=0.1)

    boxplot(ER21$mean.err ~ ER21$scene, cex=0.1)

    par(opar)
dev.off()

png 
  2

<img src="https://176.9.38.120/cruncher/files/deforestation/Img/CMASK/glm21_res_bXX_srt.png">

Несмотря на то, что численно новая модель показывает более хорошее поведение, чем simple.glm18, проблема в зависимости остатков от объясняющих переменных в новой модели не изчезла: на графиках видны все те же проблемы, что и ранее.

<a id='SnowModel3'></a>
#### Построение модели-3

Посмотрим, как будет влиять на качество модели добавление членов, описывающих взаимодействие между переменными. Начнем с модели, содержащей все члены и будем постепенно избавляться от незначащих коэффициентов.

In [59]:
simple.glm22 = glm(answ2 ~ b1 + b2 + b3 + b4 + b5 + b6 + b7 + 
                   b1:b2 + b1:b3 + b1:b4 + b1:b5 + b1:b6 + b1:b7 +
                   b2:b3 + b2:b4 + b2:b5 + b2:b6 + b2:b7 +
                   b3:b4 + b3:b5 + b3:b6 + b3:b7+
                   b4:b5 + b4:b6 + b4:b7 +
                   b5:b6 + b5:b7 +
                   b6:b7, 
                   data=pdata, family=binomial)
summary(simple.glm22)
drop1(simple.glm22, test="Chi")

Warning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurred


Call:
glm(formula = answ2 ~ b1 + b2 + b3 + b4 + b5 + b6 + b7 + b1:b2 + 
    b1:b3 + b1:b4 + b1:b5 + b1:b6 + b1:b7 + b2:b3 + b2:b4 + b2:b5 + 
    b2:b6 + b2:b7 + b3:b4 + b3:b5 + b3:b6 + b3:b7 + b4:b5 + b4:b6 + 
    b4:b7 + b5:b6 + b5:b7 + b6:b7, family = binomial, data = pdata)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-4.4512  -0.0201  -0.0042  -0.0013   4.6723  

Coefficients:
             Estimate Std. Error z value Pr(>|z|)    
(Intercept)   -27.278      3.223  -8.464  < 2e-16 ***
b1            110.583     61.932   1.786 0.074172 .  
b2            -84.763     76.813  -1.103 0.269813    
b3             57.737     80.208   0.720 0.471617    
b4             76.813     50.301   1.527 0.126741    
b5            -27.402     11.332  -2.418 0.015603 *  
b6           -386.755     95.589  -4.046 5.21e-05 ***
b7            433.011    115.792   3.740 0.000184 ***
b1:b2         113.433    153.215   0.740 0.459089    
b1:b3       -1243.721    794.490  -1.565 0.117481   

Warning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning mess

,Df,Deviance,AIC,LRT,Pr(>Chi)
<none>,NA,3866.176,3924.176,NA,NA
b1:b2,1.0000000,3866.7291358,3922.7291358,0.5528413,0.4571591
b1:b3,1.0000000,3868.6985404,3924.6985404,2.5222458,0.1122506
b1:b4,1.0000000,3868.4555489,3924.4555489,2.2792543,0.1311149
b1:b5,1.000000e+00,3.870411e+03,3.926411e+03,4.234946e+00,3.959989e-02
b1:b6,1.000000e+00,3.882187e+03,3.938187e+03,1.601057e+01,6.298984e-05
b1:b7,1.000000e+00,3.880179e+03,3.936179e+03,1.400295e+01,1.825245e-04
b2:b3,1.0000000,3867.7967063,3923.7967063,1.6204117,0.2030344
b2:b4,1.000000e+00,3.868936e+03,3.924936e+03,2.759903e+00,9.665409e-02
b2:b5,1.000000e+00,3.870635e+03,3.926635e+03,4.458361e+00,3.473081e-02


Выкинем взаимодействие b4 и b7:

In [61]:
simple.glm23 = glm(answ2 ~ b1 + b2 + b3 + b4 + b5 + b6 + b7 + 
                   b1:b2 + b1:b3 + b1:b4 + b1:b5 + b1:b6 + b1:b7 +
                   b2:b3 + b2:b4 + b2:b5 + b2:b6 + b2:b7 +
                   b3:b4 + b3:b5 + b3:b6 + b3:b7+
                   b4:b5 + b4:b6 + 
                   b5:b6 + b5:b7 +
                   b6:b7, 
                   data=pdata, family=binomial)
summary(simple.glm23)
drop1(simple.glm23, test="Chi")

Warning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurred


Call:
glm(formula = answ2 ~ b1 + b2 + b3 + b4 + b5 + b6 + b7 + b1:b2 + 
    b1:b3 + b1:b4 + b1:b5 + b1:b6 + b1:b7 + b2:b3 + b2:b4 + b2:b5 + 
    b2:b6 + b2:b7 + b3:b4 + b3:b5 + b3:b6 + b3:b7 + b4:b5 + b4:b6 + 
    b5:b6 + b5:b7 + b6:b7, family = binomial, data = pdata)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-4.4510  -0.0201  -0.0042  -0.0013   4.6711  

Coefficients:
            Estimate Std. Error z value Pr(>|z|)    
(Intercept)   -27.25       3.20  -8.515  < 2e-16 ***
b1            110.68      61.88   1.789 0.073655 .  
b2            -84.92      76.77  -1.106 0.268657    
b3             56.53      79.06   0.715 0.474641    
b4             78.03      48.44   1.611 0.107209    
b5            -27.61      11.08  -2.493 0.012666 *  
b6           -388.00      94.62  -4.101 4.12e-05 ***
b7            434.43     114.76   3.786 0.000153 ***
b1:b2         111.90     152.17   0.735 0.462132    
b1:b3       -1229.89     778.52  -1.580 0.114158    
b1:b4        1012

Warning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning mess

,Df,Deviance,AIC,LRT,Pr(>Chi)
<none>,NA,3866.184,3922.184,NA,NA
b1:b2,1.0000000,3866.7292953,3920.7292953,0.5448302,0.4604378
b1:b3,1.0000000,3868.7309681,3922.7309681,2.5465029,0.1105387
b1:b4,1.0000000,3868.5177606,3922.5177606,2.3332954,0.1266335
b1:b5,1.000000e+00,3.870469e+03,3.924469e+03,4.284563e+00,3.845998e-02
b1:b6,1.000000e+00,3.883862e+03,3.937862e+03,1.767717e+01,2.617498e-05
b1:b7,1.000000e+00,3.881677e+03,3.935677e+03,1.549285e+01,8.281815e-05
b2:b3,1.000000,3867.798419,3921.798419,1.613953,0.203937
b2:b4,1.000000e+00,3.868983e+03,3.922983e+03,2.798285e+00,9.436521e-02
b2:b5,1.0000000,3870.7075391,3924.7075391,4.5230739,0.0334407


In [71]:
simple.glm24 = glm(answ2 ~ b1 + b2 + b3 + b4 + b5 + b6 + b7 + 
                   b1:b2 + b1:b3 + b1:b4 + b1:b5 + b1:b6 + b1:b7 +
                   b2:b3 + b2:b4 + b2:b5 + b2:b6 + b2:b7 +
                   b3:b4 +         b3:b6 + b3:b7+
                   b4:b5 + b4:b6 + 
                   b5:b6 + b5:b7 +
                   b6:b7, 
                   data=pdata, family=binomial)
summary(simple.glm24)
drop1(simple.glm24, test="Chi")

Warning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurred


Call:
glm(formula = answ2 ~ b1 + b2 + b3 + b4 + b5 + b6 + b7 + b1:b2 + 
    b1:b3 + b1:b4 + b1:b5 + b1:b6 + b1:b7 + b2:b3 + b2:b4 + b2:b5 + 
    b2:b6 + b2:b7 + b3:b4 + b3:b6 + b3:b7 + b4:b5 + b4:b6 + b5:b6 + 
    b5:b7 + b6:b7, family = binomial, data = pdata)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-4.4702  -0.0201  -0.0042  -0.0013   4.6733  

Coefficients:
             Estimate Std. Error z value Pr(>|z|)    
(Intercept)   -27.208      3.175  -8.571  < 2e-16 ***
b1            107.130     60.470   1.772  0.07646 .  
b2            -76.799     72.775  -1.055  0.29129    
b3             46.865     73.693   0.636  0.52481    
b4             81.923     46.901   1.747  0.08069 .  
b5            -27.520     11.003  -2.501  0.01238 *  
b6           -388.401     94.499  -4.110 3.95e-05 ***
b7            434.567    114.615   3.792  0.00015 ***
b1:b2          99.147    147.270   0.673  0.50080    
b1:b3       -1084.735    652.319  -1.663  0.09633 .  
b1:b4         

Warning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning mess

,Df,Deviance,AIC,LRT,Pr(>Chi)
<none>,NA,3866.304,3920.304,NA,NA
b1:b2,1.0000000,3866.7599748,3918.7599748,0.4559029,0.4995450
b1:b3,1.000000e+00,3.869074e+03,3.921074e+03,2.769765e+00,9.606025e-02
b1:b4,1.0000000,3868.6804496,3920.6804496,2.3763776,0.1231829
b1:b5,1.000000e+00,3.870660e+03,3.922660e+03,4.356334e+00,3.687157e-02
b1:b6,1.000000e+00,3.884327e+03,3.936327e+03,1.802313e+01,2.182375e-05
b1:b7,1.000000e+00,3.882001e+03,3.934001e+03,1.569655e+01,7.435981e-05
b2:b3,1.0000000,3868.0073137,3920.0073137,1.7032417,0.1918646
b2:b4,1.000000e+00,3.869656e+03,3.921656e+03,3.351529e+00,6.714263e-02
b2:b5,1.000000e+00,3.870742e+03,3.922742e+03,4.437662e+00,3.515436e-02


In [72]:
simple.glm25 = glm(answ2 ~ b1 + b2 + b3 + b4 + b5 + b6 + b7 + 
                           b1:b3 + b1:b4 + b1:b5 + b1:b6 + b1:b7 +
                   b2:b3 + b2:b4 + b2:b5 + b2:b6 + b2:b7 +
                   b3:b4 +         b3:b6 + b3:b7+
                   b4:b5 + b4:b6 + 
                   b5:b6 + b5:b7 +
                   b6:b7, 
                   data=pdata, family=binomial)
summary(simple.glm25)
drop1(simple.glm25, test="Chi")

Warning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurred


Call:
glm(formula = answ2 ~ b1 + b2 + b3 + b4 + b5 + b6 + b7 + b1:b3 + 
    b1:b4 + b1:b5 + b1:b6 + b1:b7 + b2:b3 + b2:b4 + b2:b5 + b2:b6 + 
    b2:b7 + b3:b4 + +b3:b6 + b3:b7 + b4:b5 + b4:b6 + b5:b6 + 
    b5:b7 + b6:b7, family = binomial, data = pdata)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-4.4503  -0.0200  -0.0042  -0.0013   4.6730  

Coefficients:
             Estimate Std. Error z value Pr(>|z|)    
(Intercept)   -28.231      2.798 -10.090  < 2e-16 ***
b1            113.760     60.265   1.888 0.059070 .  
b2            -58.513     67.586  -0.866 0.386627    
b3             16.468     58.235   0.283 0.777337    
b4             85.498     46.607   1.834 0.066586 .  
b5            -25.017     10.285  -2.432 0.015002 *  
b6           -382.304     94.033  -4.066 4.79e-05 ***
b7            430.699    114.427   3.764 0.000167 ***
b1:b3        -965.089    628.694  -1.535 0.124767    
b1:b4         845.621    574.794   1.471 0.141244    
b1:b5        -354.168

Warning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning mess

,Df,Deviance,AIC,LRT,Pr(>Chi)
<none>,NA,3866.76,3918.76,NA,NA
b1:b3,1.0000000,3869.1229698,3919.1229698,2.3629949,0.1242435
b1:b4,1.0000000,3868.8894687,3918.8894687,2.1294939,0.1444885
b1:b5,1.000000e+00,3.870679e+03,3.920679e+03,3.919339e+00,4.773366e-02
b1:b6,1.000000e+00,3.884343e+03,3.934343e+03,1.758296e+01,2.750418e-05
b1:b7,1.000000e+00,3.882086e+03,3.932086e+03,1.532584e+01,9.047043e-05
b2:b3,1.0000000,3868.5935228,3918.5935228,1.8335480,0.1757091
b2:b4,1.000000e+00,3.869676e+03,3.919676e+03,2.916182e+00,8.769515e-02
b2:b5,1.000000e+00,3.870803e+03,3.920803e+03,4.043187e+00,4.434998e-02
b2:b6,1.000000e+00,3.876692e+03,3.926692e+03,9.932245e+00,1.624083e-03


In [73]:
simple.glm26 = glm(answ2 ~ b1 + b2 + b3 + b4 + b5 + b6 + b7 + 
                           b1:b3 + b1:b4 + b1:b5 + b1:b6 + b1:b7 +
                   b2:b3 + b2:b4 + b2:b5 + b2:b6 + b2:b7 +
                                   b3:b6 + b3:b7+
                   b4:b5 + b4:b6 + 
                   b5:b6 + b5:b7 +
                   b6:b7, 
                   data=pdata, family=binomial)
summary(simple.glm26)
drop1(simple.glm26, test="Chi")

Warning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurred


Call:
glm(formula = answ2 ~ b1 + b2 + b3 + b4 + b5 + b6 + b7 + +b1:b3 + 
    b1:b4 + b1:b5 + b1:b6 + b1:b7 + b2:b3 + b2:b4 + b2:b5 + b2:b6 + 
    b2:b7 + b3:b6 + b3:b7 + b4:b5 + b4:b6 + b5:b6 + b5:b7 + b6:b7, 
    family = binomial, data = pdata)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-4.4021  -0.0202  -0.0043  -0.0013   4.6661  

Coefficients:
             Estimate Std. Error z value Pr(>|z|)    
(Intercept)   -27.880      2.654 -10.505  < 2e-16 ***
b1            120.375     59.245   2.032 0.042173 *  
b2            -77.897     64.163  -1.214 0.224730    
b3             39.131     53.241   0.735 0.462359    
b4             77.917     46.230   1.685 0.091909 .  
b5            -27.494      9.635  -2.854 0.004323 ** 
b6           -394.217     93.302  -4.225 2.39e-05 ***
b7            448.293    113.221   3.959 7.51e-05 ***
b1:b3       -1049.421    628.814  -1.669 0.095139 .  
b1:b4         809.498    576.625   1.404 0.160361    
b1:b5        -257.386    152.

Warning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning mess

,Df,Deviance,AIC,LRT,Pr(>Chi)
<none>,NA,3868.019,3918.019,NA,NA
b1:b3,1.0000000,3870.8450601,3918.8450601,2.8263013,0.0927317
b1:b4,1.0000000,3869.9748339,3917.9748339,1.9560751,0.1619337
b1:b5,1.000000e+00,3.870747e+03,3.918747e+03,2.727912e+00,9.860813e-02
b1:b6,1.000000e+00,3.885137e+03,3.933137e+03,1.711816e+01,3.512481e-05
b1:b7,1.000000e+00,3.883301e+03,3.931301e+03,1.528259e+01,9.256558e-05
b2:b3,1.000000,3870.398028,3918.398028,2.379269,0.122955
b2:b4,1.0000000,3870.4156007,3918.4156007,2.3968419,0.1215805
b2:b5,1.000000e+00,3.870813e+03,3.918813e+03,2.794150e+00,9.460892e-02
b2:b6,1.000000e+00,3.876887e+03,3.924887e+03,8.868206e+00,2.901801e-03


In [74]:
simple.glm27 = glm(answ2 ~ b1 + b2 + b3 + b4 + b5 + b6 + b7 + 
                           b1:b3 + b1:b4 + b1:b5 + b1:b6 + b1:b7 +
                   b2:b3 + b2:b4 + b2:b5 + b2:b6 + b2:b7 +
                                    b3:b6 + b3:b7+
                           b4:b6 + 
                   b5:b6 + b5:b7 +
                   b6:b7, 
                   data=pdata, family=binomial)
summary(simple.glm27)
drop1(simple.glm27, test="Chi")

Warning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurred


Call:
glm(formula = answ2 ~ b1 + b2 + b3 + b4 + b5 + b6 + b7 + b1:b3 + 
    b1:b4 + b1:b5 + b1:b6 + b1:b7 + b2:b3 + b2:b4 + b2:b5 + b2:b6 + 
    b2:b7 + b3:b6 + b3:b7 + b4:b6 + b5:b6 + b5:b7 + b6:b7, family = binomial, 
    data = pdata)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-4.4811  -0.0203  -0.0043  -0.0013   4.6742  

Coefficients:
             Estimate Std. Error z value Pr(>|z|)    
(Intercept)   -27.635      2.648 -10.436  < 2e-16 ***
b1            104.893     56.335   1.862 0.062607 .  
b2            -54.790     58.429  -0.938 0.348389    
b3             31.392     52.739   0.595 0.551694    
b4             77.519     46.472   1.668 0.095302 .  
b5            -29.055      9.584  -3.032 0.002433 ** 
b6           -394.350     93.382  -4.223 2.41e-05 ***
b7            450.126    113.312   3.972 7.11e-05 ***
b1:b3       -1120.630    617.564  -1.815 0.069586 .  
b1:b4         853.754    571.147   1.495 0.134965    
b1:b5        -184.234    132.247  -1.3

Warning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning mess

,Df,Deviance,AIC,LRT,Pr(>Chi)
<none>,NA,3868.813,3916.813,NA,NA
b1:b3,1.000000e+00,3.872167e+03,3.918167e+03,3.353503e+00,6.706216e-02
b1:b4,1.0000000,3871.0160396,3917.0160396,2.2028431,0.1377565
b1:b5,1.0000000,3870.7473697,3916.7473697,1.9341732,0.1643026
b1:b6,1.000000e+00,3.886203e+03,3.932203e+03,1.739024e+01,3.043858e-05
b1:b7,1.000000e+00,3.885062e+03,3.931062e+03,1.624876e+01,5.554602e-05
b2:b3,1.000000e+00,3.871918e+03,3.917918e+03,3.104986e+00,7.805288e-02
b2:b4,1.0000000,3871.7105629,3917.7105629,2.8973664,0.0887244
b2:b5,1.0000000,3870.9134270,3916.9134270,2.1002305,0.1472769
b2:b6,1.000000e+00,3.878064e+03,3.924064e+03,9.250584e+00,2.354204e-03


In [75]:
simple.glm28 = glm(answ2 ~ b1 + b2 + b3 + b4 + b5 + b6 + b7 + 
                           b1:b3 + b1:b4 +          b1:b6 + b1:b7 +
                   b2:b3 + b2:b4 + b2:b5 + b2:b6 + b2:b7 +
                                   b3:b6 + b3:b7+
                           b4:b6 + 
                   b5:b6 + b5:b7 +
                   b6:b7, 
                   data=pdata, family=binomial)
summary(simple.glm28)
drop1(simple.glm28, test="Chi")

Warning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurred


Call:
glm(formula = answ2 ~ b1 + b2 + b3 + b4 + b5 + b6 + b7 + b1:b3 + 
    b1:b4 + b1:b6 + b1:b7 + b2:b3 + b2:b4 + b2:b5 + b2:b6 + b2:b7 + 
    b3:b6 + b3:b7 + b4:b6 + b5:b6 + b5:b7 + b6:b7, family = binomial, 
    data = pdata)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-4.4772  -0.0204  -0.0043  -0.0013   4.6240  

Coefficients:
             Estimate Std. Error z value Pr(>|z|)    
(Intercept)   -27.043      2.712  -9.971  < 2e-16 ***
b1             86.737     56.768   1.528 0.126531    
b2            -37.374     58.757  -0.636 0.524726    
b3             10.223     50.454   0.203 0.839438    
b4            100.548     43.434   2.315 0.020614 *  
b5            -33.760      9.284  -3.636 0.000277 ***
b6           -394.275     93.274  -4.227 2.37e-05 ***
b7            450.040    113.191   3.976 7.01e-05 ***
b1:b3        -664.799    528.779  -1.257 0.208668    
b1:b4         276.997    400.789   0.691 0.489485    
b1:b6        6212.489   1480.506   4.196 2.71e

Warning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning mess

,Df,Deviance,AIC,LRT,Pr(>Chi)
<none>,NA,3870.747,3916.747,NA,NA
b1:b3,1.0000000,3872.3713167,3916.3713167,1.6239470,0.2025423
b1:b4,1.0000000,3871.2249435,3915.2249435,0.4775738,0.4895233
b1:b6,1.000000e+00,3.887734e+03,3.931734e+03,1.698623e+01,3.765184e-05
b1:b7,1.000000e+00,3.886910e+03,3.930910e+03,1.616224e+01,5.814171e-05
b2:b3,1.0000000,3872.2045051,3916.2045051,1.4571354,0.2273865
b2:b4,1.0000000,3871.7746650,3915.7746650,1.0272953,0.3107948
b2:b5,1.0000000,3870.9248991,3914.9248991,0.1775294,0.6735051
b2:b6,1.000000e+00,3.879677e+03,3.923677e+03,8.929364e+00,2.806221e-03
b2:b7,1.000000e+00,3.879639e+03,3.923639e+03,8.891892e+00,2.864398e-03


In [76]:
simple.glm29 = glm(answ2 ~ b1 + b2 + b3 + b4 + b5 + b6 + b7 + 
                           b1:b3 + b1:b4 +          b1:b6 + b1:b7 +
                   b2:b3 + b2:b4 +         b2:b6 + b2:b7 +
                                   b3:b6 + b3:b7+
                           b4:b6 + 
                   b5:b6 + b5:b7 +
                   b6:b7, 
                   data=pdata, family=binomial)
summary(simple.glm29)
drop1(simple.glm29, test="Chi")

Warning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurred


Call:
glm(formula = answ2 ~ b1 + b2 + b3 + b4 + b5 + b6 + b7 + b1:b3 + 
    b1:b4 + b1:b6 + b1:b7 + b2:b3 + b2:b4 + b2:b6 + b2:b7 + b3:b6 + 
    b3:b7 + b4:b6 + b5:b6 + b5:b7 + b6:b7, family = binomial, 
    data = pdata)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-4.4549  -0.0203  -0.0043  -0.0013   4.6141  

Coefficients:
             Estimate Std. Error z value Pr(>|z|)    
(Intercept)   -27.728      2.217 -12.508  < 2e-16 ***
b1             94.808     54.444   1.741 0.081617 .  
b2            -44.803     56.823  -0.788 0.430422    
b3             18.889     46.078   0.410 0.681856    
b4             92.145     38.544   2.391 0.016818 *  
b5            -30.457      5.123  -5.946 2.75e-09 ***
b6           -384.416     90.517  -4.247 2.17e-05 ***
b7            436.976    109.193   4.002 6.28e-05 ***
b1:b3        -741.330    499.490  -1.484 0.137762    
b1:b4         319.658    389.606   0.820 0.411951    
b1:b6        6033.430   1420.333   4.248 2.16e-05 ***


Warning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning mess

,Df,Deviance,AIC,LRT,Pr(>Chi)
<none>,NA,3870.925,3914.925,NA,NA
b1:b3,1.0000000,3873.1378988,3915.1378988,2.2129997,0.1368523
b1:b4,1.0000000,3871.5941222,3913.5941222,0.6692231,0.4133226
b1:b6,1.000000e+00,3.888397e+03,3.930397e+03,1.747205e+01,2.915622e-05
b1:b7,1.000000e+00,3.887508e+03,3.929508e+03,1.658322e+01,4.656112e-05
b2:b3,1.0000000,3872.7419010,3914.7419010,1.8170019,0.1776706
b2:b4,1.0000000,3872.0501138,3914.0501138,1.1252147,0.2887984
b2:b6,1.000000e+00,3.879813e+03,3.921813e+03,8.888473e+00,2.869766e-03
b2:b7,1.000000e+00,3.879719e+03,3.921719e+03,8.794342e+00,3.021662e-03
b3:b6,1.0000000,3876.2377981,3918.2377981,5.3128990,0.0211681


In [68]:
simple.glm30 = glm(answ2 ~ b1 + b2 + b3 + b4 + b5 + b6 + b7 + 
                           b1:b3 +         b1:b6 + b1:b7 +
                   b2:b3 + b2:b4 +         b2:b6 + b2:b7 +
                                   b3:b6 + b3:b7+
                           b4:b6 + 
                   b5:b6 + b5:b7 +
                   b6:b7, 
                   data=pdata, family=binomial)
summary(simple.glm30)
drop1(simple.glm30, test="Chi")

Warning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurred


Call:
glm(formula = answ2 ~ b1 + b2 + b3 + b4 + b5 + b6 + b7 + +b1:b3 + 
    b1:b6 + b1:b7 + b2:b3 + b2:b4 + b2:b6 + b2:b7 + b3:b6 + b3:b7 + 
    b4:b6 + b5:b6 + b5:b7 + b6:b7, family = binomial, data = pdata)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-4.6181  -0.0205  -0.0043  -0.0013   4.6103  

Coefficients:
             Estimate Std. Error z value Pr(>|z|)    
(Intercept)   -27.055      2.045 -13.228  < 2e-16 ***
b1             75.686     48.513   1.560 0.118736    
b2            -25.564     51.130  -0.500 0.617082    
b3              5.723     42.836   0.134 0.893717    
b4            102.164     36.238   2.819 0.004814 ** 
b5            -30.909      5.078  -6.087 1.15e-09 ***
b6           -390.731     90.072  -4.338 1.44e-05 ***
b7            441.502    108.906   4.054 5.04e-05 ***
b1:b3        -347.468    132.789  -2.617 0.008879 ** 
b1:b6        6243.856   1399.967   4.460 8.20e-06 ***
b1:b7       -7226.277   1695.947  -4.261 2.04e-05 ***
b2:b3       

Warning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurred

,Df,Deviance,AIC,LRT,Pr(>Chi)
<none>,NA,3871.594,3913.594,NA,NA
b1:b3,1.000000e+00,3.880972e+03,3.920972e+03,9.377737e+00,2.196365e-03
b1:b6,1.000000e+00,3.890858e+03,3.930858e+03,1.926417e+01,1.138222e-05
b1:b7,1.000000e+00,3.889298e+03,3.929298e+03,1.770390e+01,2.580970e-05
b2:b3,1.000000e+00,3.874307e+03,3.914307e+03,2.713121e+00,9.952609e-02
b2:b4,1.0000000,3873.2736451,3913.2736451,1.6795229,0.1949879
b2:b6,1.000000e+00,3.881522e+03,3.921522e+03,9.927394e+00,1.628369e-03
b2:b7,1.000000e+00,3.881009e+03,3.921009e+03,9.415305e+00,2.151817e-03
b3:b6,1.000000e+00,3.876695e+03,3.916695e+03,5.100386e+00,2.392052e-02
b3:b7,1.000000e+00,3.877991e+03,3.917991e+03,6.396965e+00,1.143156e-02


In [77]:
simple.glm31 = glm(answ2 ~ b1 + b2 + b3 + b4 + b5 + b6 + b7 + 
                           b1:b3 +         b1:b6 + b1:b7 +
                   b2:b3 +         b2:b6 + b2:b7 +
                                   b3:b6 + b3:b7+
                           b4:b6 + 
                   b5:b6 + b5:b7 +
                   b6:b7, 
                   data=pdata, family=binomial)
summary(simple.glm31)
drop1(simple.glm31, test="Chi")

Warning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurred


Call:
glm(formula = answ2 ~ b1 + b2 + b3 + b4 + b5 + b6 + b7 + b1:b3 + 
    b1:b6 + b1:b7 + b2:b3 + b2:b6 + b2:b7 + b3:b6 + b3:b7 + b4:b6 + 
    b5:b6 + b5:b7 + b6:b7, family = binomial, data = pdata)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-4.7820  -0.0213  -0.0046  -0.0014   4.5718  

Coefficients:
             Estimate Std. Error z value Pr(>|z|)    
(Intercept)   -26.905      1.908 -14.099  < 2e-16 ***
b1             49.157     36.630   1.342 0.179607    
b2              2.305     41.182   0.056 0.955358    
b3             41.872     27.307   1.533 0.125179    
b4             64.454     17.285   3.729 0.000192 ***
b5            -29.799      4.839  -6.158 7.39e-10 ***
b6           -406.420     85.289  -4.765 1.89e-06 ***
b7            465.817    101.209   4.603 4.17e-06 ***
b1:b3        -234.118     86.638  -2.702 0.006887 ** 
b1:b6        6535.895   1370.104   4.770 1.84e-06 ***
b1:b7       -7687.165   1634.561  -4.703 2.57e-06 ***
b2:b3          81.669

Warning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurred

,Df,Deviance,AIC,LRT,Pr(>Chi)
<none>,NA,3873.274,3913.274,NA,NA
b1:b3,1.000000e+00,3.881385e+03,3.919385e+03,8.111390e+00,4.398799e-03
b1:b6,1.000000e+00,3.895842e+03,3.933842e+03,2.256818e+01,2.028162e-06
b1:b7,1.000000e+00,3.895738e+03,3.933738e+03,2.246388e+01,2.141327e-06
b2:b3,1.0000000,3874.4045223,3912.4045223,1.1308771,0.2875883
b2:b6,1.000000e+00,3.884545e+03,3.922545e+03,1.127126e+01,7.871649e-04
b2:b7,1.000000e+00,3.884374e+03,3.922374e+03,1.110009e+01,8.632366e-04
b3:b6,1.000000e+00,3.879190e+03,3.917190e+03,5.916294e+00,1.500148e-02
b3:b7,1.000000e+00,3.882873e+03,3.920873e+03,9.598942e+00,1.946895e-03
b4:b6,1.000000e+00,3.882673e+03,3.920673e+03,9.399606e+00,2.170320e-03


In [78]:
simple.glm32 = glm(answ2 ~ b1 + b2 + b3 + b4 + b5 + b6 + b7 + 
                           b1:b3 +         b1:b6 + b1:b7 +
                                   b2:b6 + b2:b7 +
                                   b3:b6 + b3:b7+
                           b4:b6 + 
                   b5:b6 + b5:b7 +
                   b6:b7, 
                   data=pdata, family=binomial)
summary(simple.glm32)
drop1(simple.glm32, test="Chi")

Warning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurred


Call:
glm(formula = answ2 ~ b1 + b2 + b3 + b4 + b5 + b6 + b7 + b1:b3 + 
    b1:b6 + b1:b7 + b2:b6 + b2:b7 + b3:b6 + b3:b7 + b4:b6 + b5:b6 + 
    b5:b7 + b6:b7, family = binomial, data = pdata)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-4.5997  -0.0211  -0.0045  -0.0014   4.5748  

Coefficients:
             Estimate Std. Error z value Pr(>|z|)    
(Intercept)   -26.023      1.682 -15.473  < 2e-16 ***
b1             29.625     29.427   1.007 0.314070    
b2             21.293     35.594   0.598 0.549690    
b3             34.736     25.898   1.341 0.179836    
b4             65.834     16.956   3.883 0.000103 ***
b5            -29.578      4.779  -6.190 6.03e-10 ***
b6           -441.624     76.501  -5.773 7.80e-09 ***
b7            510.671     88.710   5.757 8.58e-09 ***
b1:b3        -144.494      8.057 -17.934  < 2e-16 ***
b1:b6        7153.253   1231.782   5.807 6.35e-09 ***
b1:b7       -8498.161   1433.141  -5.930 3.03e-09 ***
b2:b6       -5533.573   1379.

Warning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurred

,Df,Deviance,AIC,LRT,Pr(>Chi)
<none>,NA,3874.405,3912.405,NA,NA
b1:b3,1.000000e+00,4.128872e+03,4.164872e+03,2.544673e+02,2.757785e-57
b1:b6,1.000000e+00,3.908466e+03,3.944466e+03,3.406170e+01,5.339184e-09
b1:b7,1.000000e+00,3.910565e+03,3.946565e+03,3.616032e+01,1.817337e-09
b2:b6,1.000000e+00,3.889894e+03,3.925894e+03,1.548929e+01,8.297409e-05
b2:b7,1.000000e+00,3.890130e+03,3.926130e+03,1.572590e+01,7.321467e-05
b3:b6,1.000000e+00,3.881290e+03,3.917290e+03,6.885112e+00,8.691664e-03
b3:b7,1.000000e+00,3.885605e+03,3.921605e+03,1.120096e+01,8.175490e-04
b4:b6,1.000000e+00,3.885247e+03,3.921247e+03,1.084249e+01,9.919722e-04
b5:b6,1.000000e+00,3.927467e+03,3.963467e+03,5.306237e+01,3.231242e-13


Пришли к тому моменту, когда нет возможности выкинуть незначимые члены. Заметим, что команда `summary(simple.glm32)` говорит о том, что у коэффициенты перед b1, b2, b3 незначимы, но мы не можем убрать эти переменные, поскольку они учавствуют во взаимодействиях сдругими переменными.

Поэтому остановимся на данной модели и посмотрим, как ведут себя ее остатки.

<a id='SnowValidation3'></a>
##### Валидация модели-3

In [79]:
E32 = resid(simple.glm32, type="response")
ER32 = data.frame(err=E32, fitted=simple.glm32$fitted.values, 
                b1=pdata$b1, b2=pdata$b2, b3=pdata$b3, b4=pdata$b4, b5=pdata$b5, b6=pdata$b6, 
                b7=pdata$b7, b9=pdata$b9, b10=pdata$b10, b11=pdata$b11, scene=pdata$scene)

# Сортируем по порядку прогнозируемых величин
ER32 = ER32[order(ER32$fitted), ]

# Разбиваем на группы
group_size = 100
ER32 = cbind(id=(1:length(ER32$fitted))%/% group_size, ER32)
# Вычислим среднее значение ошибки в каждой группе и поместим в таблицу
lookup_table = aggregate(ER32['err'], by=list(ER32$id), FUN=mean)
names(lookup_table) = c('id', 'err')

# Создадим вектор из средний значений ошибок 
a = sapply(ER32$id, function(id){lookup_table[lookup_table$id==id, 'err']})

ER32 = cbind(ER32, mean.err=a)


png("Img/CMASK/glm32_res_fitted_srt.png", width=728, height=728, units="px")
  plot(ER32$fitted, ER32$mean.err, cex=0.1)
dev.off()

png 
  2

<img src="https://176.9.38.120/cruncher/files/deforestation/Img/CMASK/glm32_res_fitted_srt.png">

На этом графике принципиальных улучшений не видно.

Посмотрим графики зависимостей остатков от объясняющих переменных.

In [80]:
png("Img/CMASK/glm32_res_bXX_srt.png", width=728, height=728, units="px")
    par(mfrow = c(3, 4))
    plot(ER32$b1, ER32$mean.err, cex=0.1)
    plot(ER32$b2, ER32$mean.err, cex=0.1)
    plot(ER32$b3, ER32$mean.err, cex=0.1)
    plot(ER32$b4, ER32$mean.err, cex=0.1)

    plot(ER32$b5, ER32$mean.err, cex=0.1)
    plot(ER32$b6, ER32$mean.err, cex=0.1)
    plot(ER32$b7, ER32$mean.err, cex=0.1)

    plot(ER32$b9, ER32$mean.err, cex=0.1)

    plot(ER32$b10, ER32$mean.err, cex=0.1)
    plot(ER32$b11, ER32$mean.err, cex=0.1)

    boxplot(ER32$mean.err ~ ER32$scene, cex=0.1)

    par(opar)
dev.off()

png 
  2

<img src="https://176.9.38.120/cruncher/files/deforestation/Img/CMASK/glm32_res_bXX_srt.png">

На графиках видно, что закономерности в распределении остатков не исчезли, например, видна тенденция к низким значениям отстаков при низких значениях b4. У части других переменных та же самая проблема.

Таким образом, несмотря на улучшившийся AIC, снизившийся до 3912.4, нельзя сказать, что данная модель дает хорошее описание классов снег-не-снег.

<a id='SnowResume'></a>
##### Вывод

В этом разделе были построены модели логистической регрессии с различными вариантами объясняющих переменных. Наиболее точной моделью является модель `simple.glm32`. К сожалению, все модели, в том числе и `simple.glm32` показывают определенные закономерности в распределении остатков. Это говорит о том, что:

 * либо выбрана неверная статистическая модель;
 * либо выбранные объясняющие переменные не позволяют качественно отделить классы.
 
Таким образом следует или дальше пытаться подобрать переменные, включая в модель степенные члены и члены с их взаимодействием, или строить классификатор на базе других статистических моделей, а не логистической регрессии. Первый путь сопряжен с большим объемом работ, поэтому в следующем разделе будет представлена попытка построить аддитивную модель.

In [35]:
library(mgcv)

Loading required package: nlme
This is mgcv 1.7-28. For overview type 'help("mgcv-package")'.
